In [11]:
import pandas as pd

# Load Files
Acquisitions = pd.read_csv("../data/acquisitions.csv")
Degrees = pd.read_csv("../data/degrees.csv")
Ipos = pd.read_csv("../data/ipos.csv")
Investments = pd.read_csv("../data/investments.csv")
Milestones = pd.read_csv("../data/milestones.csv")
Relationships = pd.read_csv("../data/relationships.csv")
People = pd.read_csv("../data/people.csv")

print("="*60)
print("Dataset 로드 완료!")
print("="*60)

Dataset 로드 완료!


## Acquisitions 데이터 전처리

#### Acqusitions.price_amount 0값 처리

In [12]:
# acquisitions ←> milestone merge
acquisitions_milesone = (
    Acquisitions
    .merge(Milestones, how='left', left_on='acquired_object_id', right_on='object_id')
)

### 1) acquired milestone.description 텍스트 분석

In [13]:
import pandas as pd
import re

# ------------------------------------------------------------
# 1) 전처리: price_amount 숫자형으로 맞추기
# ------------------------------------------------------------
Acquisitions_copy = Acquisitions.copy()
Acquisitions_copy["price_amount"] = pd.to_numeric(Acquisitions_copy["price_amount"], errors="coerce")

tmp = acquisitions_milesone.copy()
tmp["desc"] = tmp["description"].fillna("").str.lower()

# ------------------------------------------------------------
# 2) (A) 비공개(undisclosed) 근거 탐지  -> is_acqusitions_price_private
# ------------------------------------------------------------
UNDISCLOSED_PATTERN = r"""
(
    undisclosed(\s+(amount|sum|price|fee|consideration|valuation|deal\s*value|terms))?
  | amount\s+undisclosed
  | not\s+disclosed
  | terms\s+were\s+not\s+disclosed
  | financial\s+terms\s+were\s+not\s+disclosed
  | deal\s+terms\s+were\s+not\s+disclosed
  | price\s+was\s+not\s+disclosed
  | purchase\s+price\s+was\s+not\s+disclosed
  | transaction\s+value\s+was\s+not\s+disclosed
  | undisclosed\s+financial\s+(terms|details)
)
"""
undisclosed_re = re.compile(UNDISCLOSED_PATTERN, flags=re.IGNORECASE | re.VERBOSE)

tmp["is_undisclosed_in_milestone"] = tmp["description"].fillna("").apply(
    lambda x: bool(undisclosed_re.search(x))
)

undisclosed_by_acquired = (
    tmp.groupby("acquired_object_id")["is_undisclosed_in_milestone"]
       .any()
       .rename("is_acqusitions_price_private")
       .reset_index()
)

# ------------------------------------------------------------
# 3) (B) 흡수/통합/종료 후 흡수(absorbed 등) 탐지 -> is_absorbed_deal
#    - "absorbed" 자체만이 아니라, 실제로 자주 나오는 표현들을 폭넓게 잡음
# ------------------------------------------------------------
ABSORBED_PATTERN = r"""
(
    absorbed
  | acqui[-\s]?hire
  | talent\s+acquisition
  | (shut\s*down|closed\s*down)\s*;?\s*(absorbed|folded\s+into|integrated\s+into)
  | folded\s+into
  | rolled\s+into
  | integrated\s+into
  | merged\s+into
  | consolidated\s+into
  | combined\s+with
  | joined\s+with
  | incorporated\s+into
  | became\s+part\s+of
  | rebranded\s+as
)
"""
absorbed_re = re.compile(ABSORBED_PATTERN, flags=re.IGNORECASE | re.VERBOSE)

tmp["is_absorbed_in_milestone"] = tmp["description"].fillna("").apply(
    lambda x: bool(absorbed_re.search(x))
)

absorbed_by_acquired = (
    tmp.groupby("acquired_object_id")["is_absorbed_in_milestone"]
       .any()
       .rename("is_absorbed_deal")
       .reset_index()
)

# ------------------------------------------------------------
# 4) acquisitions 테이블에 플래그 붙이기 (비공개 + 흡수)
# ------------------------------------------------------------
Acquisitions_copy = Acquisitions_copy.merge(
    undisclosed_by_acquired,
    on="acquired_object_id",
    how="left"
).merge(
    absorbed_by_acquired,
    on="acquired_object_id",
    how="left"
)

Acquisitions_copy["is_acqusitions_price_private"] = (
    Acquisitions_copy["is_acqusitions_price_private"].fillna(False).astype(int)
)

Acquisitions_copy["is_absorbed_deal"] = (
    Acquisitions_copy["is_absorbed_deal"].fillna(False).astype(int)
)

# ------------------------------------------------------------
# 5) price_amount 처리 규칙 (최종 정답 로직)
#    - 비공개면: price_amount == 0 → NaN
#    - 흡수형은: 0 그대로 유지 (진짜 0)
# ------------------------------------------------------------

mask_private = (
    (Acquisitions_copy["price_amount"] == 0) &
    (Acquisitions_copy["is_acqusitions_price_private"] == 1)
)

Acquisitions_copy.loc[mask_private, "price_amount"] = pd.NA

# ------------------------------------------------------------
# 6) 확인용
# ------------------------------------------------------------
print("비공개 플래그 분포:\n", Acquisitions_copy["is_acqusitions_price_private"].value_counts(dropna=False))
print("흡수형 플래그 분포:\n", Acquisitions_copy["is_absorbed_deal"].value_counts(dropna=False))
print("price_amount==0 개수(진짜 0로 남은 것):", (Acquisitions_copy["price_amount"] == 0).sum())
print("price_amount NaN 개수(비공개/흡수 처리된 것):", Acquisitions_copy["price_amount"].isna().sum())


비공개 플래그 분포:
 is_acqusitions_price_private
0    9547
1      15
Name: count, dtype: int64
흡수형 플래그 분포:
 is_absorbed_deal
0    9553
1       9
Name: count, dtype: int64
price_amount==0 개수(진짜 0로 남은 것): 6953
price_amount NaN 개수(비공개/흡수 처리된 것): 10


/var/folders/59/zj68531j46j58_ylcbpsyxl80000gn/T/ipykernel_31625/1028694003.py:92: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Acquisitions_copy["is_acqusitions_price_private"].fillna(False).astype(int)
/var/folders/59/zj68531j46j58_ylcbpsyxl80000gn/T/ipykernel_31625/1028694003.py:96: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Acquisitions_copy["is_absorbed_deal"].fillna(False).astype(int)


### 2) 인수 직후 핵심 인력이 인수사로 이동

In [14]:
Relationships

,id,relationship_id,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title,created_at,updated_at
0,1,1,p:2,c:1,NaN,NaN,0,8,Co-Founder/CEO/Board of Directors,2007-05-25 07:03:54,2013-06-03 09:58:46
1,2,2,p:3,c:1,NaN,NaN,1,279242,VP Marketing,2007-05-25 07:04:16,2010-05-21 16:31:34
2,3,3,p:4,c:3,NaN,NaN,0,4,Evangelist,2007-05-25 19:33:03,2013-06-29 13:36:58
3,4,4,p:5,c:3,2006-03-01,2009-12-01,1,4,Senior Director Strategic Alliances,2007-05-25 19:34:53,2013-06-29 10:25:34
4,6,6,p:7,c:4,2005-07-01,2010-04-05,1,1,Chief Executive Officer,2007-05-25 20:05:33,2010-04-05 18:41:41
...,...,...,...,...,...,...,...,...,...,...,...
402873,480903,480903,p:268527,c:286218,2005-01-01,2010-01-01,1,1,Sr. Consultant/Patent Agent,2013-12-12 14:35:35,2013-12-12 14:35:35
402874,480906,480906,p:268597,c:12612,NaN,NaN,0,1,"VP, GM & Editor In Chief - Clinical Decison Su...",2013-12-12 14:37:56,2013-12-12 14:37:56
402875,480907,480907,p:268528,c:286152,NaN,NaN,1,273049,Vice President,2013-12-12 14:39:36,2013-12-12 14:39:36
402876,480908,480908,p:268528,c:286220,NaN,NaN,1,1,"General Counsel, Director of Patents and Licen...",2013-12-12 14:39:48,2013-12-12 14:39:48


In [16]:
# ============================================================
# 7) Relationships / People 기반 "흡수형(acqui-hire)" 보조 신호
# ============================================================

import numpy as np

# -----------------------------
# (1) 파라미터 설정
# -----------------------------
WINDOW_DAYS = 180    # 인수일 기준 ±180일
MIN_MOVERS = 3       # 이동 인원 기준 (1~5로 조정 가능)

# -----------------------------
# (2) 날짜 타입 정리
# -----------------------------
rels = Relationships.copy()
rels["start_at"] = pd.to_datetime(rels["start_at"], errors="coerce")
rels["end_at"]   = pd.to_datetime(rels["end_at"], errors="coerce")

acq = Acquisitions_copy.copy()
acq["acquired_at"] = pd.to_datetime(acq["acquired_at"], errors="coerce")

acq = acq[acq["acquired_at"].notna()].copy()

# -----------------------------
# (3) 인수 이벤트별 인력 이동 탐색
# -----------------------------
result_rows = []

for row in acq[["acquired_object_id", "acquiring_object_id", "acquired_at"]].itertuples(index=False):
    acquired_id  = row.acquired_object_id
    acquiring_id = row.acquiring_object_id
    t0 = row.acquired_at

    t_before = t0 - pd.Timedelta(days=WINDOW_DAYS)
    t_after  = t0 + pd.Timedelta(days=WINDOW_DAYS)

    # 피인수사에서 관계 종료된 사람
    left_acquired = rels[
        (rels["relationship_object_id"] == acquired_id) &
        (rels["end_at"].notna()) &
        (rels["end_at"].between(t_before, t_after))
    ][["person_object_id"]]

    if left_acquired.empty:
        result_rows.append((acquired_id, acquiring_id, 0))
        continue

    # 인수사에서 관계 시작한 사람
    joined_acquiring = rels[
        (rels["relationship_object_id"] == acquiring_id) &
        (rels["start_at"].notna()) &
        (rels["start_at"].between(t0, t_after))
    ][["person_object_id"]]

    if joined_acquiring.empty:
        result_rows.append((acquired_id, acquiring_id, 0))
        continue

    movers = left_acquired.merge(joined_acquiring, on="person_object_id", how="inner")
    n_movers = movers["person_object_id"].nunique()

    result_rows.append((acquired_id, acquiring_id, n_movers))

# -----------------------------
# (4) 요약 테이블 생성
# -----------------------------
mover_summary = pd.DataFrame(
    result_rows,
    columns=["acquired_object_id", "acquiring_object_id", "n_movers_180d"]
).drop_duplicates()

# -----------------------------
# (5) Acquisitions_copy에 병합
# -----------------------------
Acquisitions_copy = Acquisitions_copy.merge(
    mover_summary,
    on=["acquired_object_id", "acquiring_object_id"],
    how="left"
)

Acquisitions_copy["n_movers_180d"] = (
    Acquisitions_copy["n_movers_180d"].fillna(0).astype(int)
)

# -----------------------------
# (6) 흡수형 플래그 생성
# -----------------------------
Acquisitions_copy["is_absorbed_signal"] = (
    Acquisitions_copy["n_movers_180d"] >= MIN_MOVERS
).astype(int)

# -----------------------------
# (7) 확인
# -----------------------------
print("흡수형 신호 분포:\n", Acquisitions_copy["is_absorbed_signal"].value_counts(dropna=False))
print(Acquisitions_copy[["n_movers_180d"]].describe())


흡수형 신호 분포:
 is_absorbed_signal
0    9554
1       8
Name: count, dtype: int64
       n_movers_180d
count    9562.000000
mean        0.028341
std         0.205387
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000


### 3) acquired milestone.description 텍스트 분석

In [17]:
# acquisitions ←> milestone merge
acquisitions_milesone = (
    Acquisitions
    .merge(Milestones, how='left', left_on='acquiring_object_id', right_on='object_id')
)

In [18]:
# ============================================================
# 8) (인수사 기준) Milestones description으로
#    - 흡수 텍스트 플래그 (is_absorbed_text)
#    - 비공개 텍스트 플래그 (is_price_private_text)
#    를 만들고 Acquisitions_copy에 붙이기
# ============================================================

# 텍스트 준비
tmp2 = acquisitions_milesone.copy()
tmp2["desc"] = tmp2["description"].fillna("").str.lower()

# ------------------------------------------------------------
# 1) 흡수(흡수형/acqui-hire/통합) 텍스트 패턴
# ------------------------------------------------------------
ABSORBED_PATTERN = r"""
(
    \babsorbed\b
  | \bfolded\s+into\b
  | \brolled\s+into\b
  | \bintegrated\s+into\b
  | \bmerged\s+into\b
  | \bconsolidated\s+into\b
  | \bincorporated\s+into\b
  | \bbecame\s+part\s+of\b
  | \bacqui[-\s]?hire\b
  | \btalent\s+acquisition\b
)
"""
absorbed_re = re.compile(ABSORBED_PATTERN, flags=re.IGNORECASE | re.VERBOSE)

tmp2["is_absorbed_text_row"] = tmp2["desc"].apply(lambda x: bool(absorbed_re.search(x)))

# ------------------------------------------------------------
# 2) 비공개(undisclosed / not disclosed 등) 텍스트 패턴
# ------------------------------------------------------------
PRIVATE_PATTERN = r"""
(
    \bundisclosed\b
  | not\s+(publicly\s+)?disclosed
  | (amount|price|terms|details)\s+were?\s+not\s+disclosed
  | (amount|price|terms|details)\s+not\s+disclosed
  | price\s+was\s+not\s+disclosed
  | purchase\s+price\s+was\s+not\s+disclosed
  | transaction\s+value\s+was\s+not\s+disclosed
  | financial\s+terms\s+were\s+not\s+disclosed
  | deal\s+terms\s+were\s+not\s+disclosed
  | declined\s+to\s+disclose
  | chose\s+not\s+to\s+disclose
  | refused\s+to\s+disclose
)
"""
private_re = re.compile(PRIVATE_PATTERN, flags=re.IGNORECASE | re.VERBOSE)

tmp2["is_price_private_text_row"] = tmp2["desc"].apply(lambda x: bool(private_re.search(x)))

# ------------------------------------------------------------
# 3) acquiring_object_id 단위로 "하나라도 있으면 True"로 집계
# ------------------------------------------------------------
absorbed_text_by_acquirer = (
    tmp2.groupby("acquiring_object_id")["is_absorbed_text_row"]
        .any()
        .rename("is_absorbed_text")
        .reset_index()
)

private_text_by_acquirer = (
    tmp2.groupby("acquiring_object_id")["is_price_private_text_row"]
        .any()
        .rename("is_price_private_text")
        .reset_index()
)

Acquisitions_copy = Acquisitions_copy.drop(
    columns=[
        "is_absorbed_text",
        "is_price_private_text",
        "is_absorbed_final",
        "is_price_private_final",
    ],
    errors="ignore"
)

# ------------------------------------------------------------
# 4) Acquisitions_copy에 붙이기
# ------------------------------------------------------------
Acquisitions_copy = (
    Acquisitions_copy
    .merge(absorbed_text_by_acquirer, on="acquiring_object_id", how="left")
    .merge(private_text_by_acquirer, on="acquiring_object_id", how="left")
)

Acquisitions_copy["is_absorbed_text"] = Acquisitions_copy["is_absorbed_text"].fillna(False).astype(int)
Acquisitions_copy["is_price_private_text"] = Acquisitions_copy["is_price_private_text"].fillna(False).astype(int)

# ------------------------------------------------------------
# 5) (선택) 최종 흡수/비공개 플래그 만들기
#    - 흡수: 인력 이동 OR 텍스트
#    - 비공개: 텍스트 기반 (너가 앞에서 만든 milestone 비공개 플래그가 있으면 OR로 합쳐도 됨)
# ------------------------------------------------------------
Acquisitions_copy["is_absorbed_final"] = (
    (Acquisitions_copy["is_absorbed_signal"] == 1) |
    (Acquisitions_copy["is_absorbed_text"] == 1)
).astype(int)

Acquisitions_copy["is_price_private_final"] = (
    (Acquisitions_copy["is_price_private_text"] == 1) |
    (Acquisitions_copy.get("is_acqusitions_price_private", 0) == 1)  # 기존 비공개 플래그가 있으면 같이 반영
).astype(int)

# ------------------------------------------------------------
# 6) 확인
# ------------------------------------------------------------
print("\n[CHECK] 흡수 텍스트 플래그 분포(is_absorbed_text)")
print(Acquisitions_copy["is_absorbed_text"].value_counts(dropna=False))

print("\n[CHECK] 비공개 텍스트 플래그 분포(is_price_private_text)")
print(Acquisitions_copy["is_price_private_text"].value_counts(dropna=False))

print("\n[CHECK] 최종 흡수 플래그 분포(is_absorbed_final)")
print(Acquisitions_copy["is_absorbed_final"].value_counts(dropna=False))

print("\n[CHECK] 최종 비공개 플래그 분포(is_price_private_final)")
print(Acquisitions_copy["is_price_private_final"].value_counts(dropna=False))



[CHECK] 흡수 텍스트 플래그 분포(is_absorbed_text)
is_absorbed_text
0    9428
1     134
Name: count, dtype: int64

[CHECK] 비공개 텍스트 플래그 분포(is_price_private_text)
is_price_private_text
0    9391
1     171
Name: count, dtype: int64

[CHECK] 최종 흡수 플래그 분포(is_absorbed_final)
is_absorbed_final
0    9421
1     141
Name: count, dtype: int64

[CHECK] 최종 비공개 플래그 분포(is_price_private_final)
is_price_private_final
0    9377
1     185
Name: count, dtype: int64


### 4) 생성 컬럼 합치고 비공개 컬럼만 생성

In [19]:
import pandas as pd
import numpy as np

Acquisitions_copy = Acquisitions_copy.copy()

# ------------------------------------------------------------
# 1) 흡수 여부를 "임시로만" 계산 (컬럼으로 남기지 않음)
#    - 존재하는 흡수 신호 컬럼들만 OR로 합침
# ------------------------------------------------------------
absorbed_cols = ["is_absorbed_signal", "is_absorbed_text", "is_absorbed_deal"]
absorbed_exist = [c for c in absorbed_cols if c in Acquisitions_copy.columns]

if len(absorbed_exist) == 0:
    absorbed_mask = pd.Series(False, index=Acquisitions_copy.index)
else:
    absorbed_mask = Acquisitions_copy[absorbed_exist].fillna(0).astype(int).max(axis=1).astype(bool)

# 흡수로 판단된 애들은 price_amount를 0으로 확정 (NaN/다른 값이 있더라도 0으로)
Acquisitions_copy.loc[absorbed_mask, "price_amount"] = 0

# ------------------------------------------------------------
# 2) 비공개 플래그 1개만 생성
#    - "흡수 아닌데 price_amount==0" => 비공개
# ------------------------------------------------------------
Acquisitions_copy["is_acq_price_private"] = ((~absorbed_mask) & (Acquisitions_copy["price_amount"] == 0)).astype(int)

# 비공개는 price_amount를 NaN 처리
Acquisitions_copy.loc[Acquisitions_copy["is_acq_price_private"] == 1, "price_amount"] = pd.NA

# ------------------------------------------------------------
# 3) 중간 플래그 컬럼/중간 계산 컬럼 전부 삭제
# ------------------------------------------------------------
drop_cols = [
    "is_absorbed_signal", "is_absorbed_text", "is_absorbed_deal",
    "is_price_private_text", "is_acqusitions_price_private",
    "is_absorbed_final", "is_price_private_final",
    "n_movers_180d"
]
Acquisitions_copy = Acquisitions_copy.drop(columns=[c for c in drop_cols if c in Acquisitions_copy.columns])

# ------------------------------------------------------------
# 4) 확인
# ------------------------------------------------------------
print("[CHECK] is_acq_price_private 분포")
print(Acquisitions_copy["is_acq_price_private"].value_counts(dropna=False))

print("\n[CHECK] price_amount==0 개수(=흡수로 남은 0)")
print((Acquisitions_copy["price_amount"] == 0).sum())

print("\n[CHECK] price_amount NaN 개수(=비공개 처리)")
print(Acquisitions_copy["price_amount"].isna().sum())

Acquisitions_final = Acquisitions_copy


[CHECK] is_acq_price_private 분포
is_acq_price_private
1    6850
0    2712
Name: count, dtype: int64

[CHECK] price_amount==0 개수(=흡수로 남은 0)
150

[CHECK] price_amount NaN 개수(=비공개 처리)
6860


In [20]:
# 컬럼 제거
Acquisitions_copy = Acquisitions_copy.drop(columns=['id', 'source_url', 'source_description', 'created_at', 'updated_at'])

In [21]:
# acquiring_object_id, acquired_object_id 컬럼명 변경
alter_columns = Acquisitions_copy.rename(columns={
    'acquiring_object_id': 'acquiring_c_id',
    'acquired_object_id': 'acquired_c_id'
}, inplace=True)

In [22]:
# acquired_at 형 변환
Acquisitions_copy['acquired_at'] = pd.to_datetime(Acquisitions_copy['acquired_at'])

# acquired_at 결측값 플래그 컬럼 생성
Acquisitions_copy["is_acquisitions_acq_at_missing"] = Acquisitions_copy["acquired_at"].isna().astype(int)

### 환율 계산

In [27]:
# Load Files (경로확인 후 실행!)
usd_aud = pd.read_csv("../환율 계산/USD_AUD_rate.csv")
usd_cad = pd.read_csv("../환율 계산/USD_CAD_rate.csv")
usd_eur = pd.read_csv("../환율 계산/USD_EUR_rate.csv")
usd_gbp = pd.read_csv("../환율 계산/USD_GBP_rate.csv")
usd_jpy = pd.read_csv("../환율 계산/USD_JPY_rate.csv")
usd_sek = pd.read_csv("../환율 계산/USD_SEK_rate.csv")

print("="*60)
print("환율 Dataset 로드 완료!")
print("="*60)

환율 Dataset 로드 완료!


In [28]:
# 각 환율코드별 날짜 min&max 값 찾기 
print("="*60)
print("AUD")
cond1 = (Acquisitions_copy['price_currency_code'] =="AUD")
print(Acquisitions_copy.loc[cond1, "acquired_at"].min())
print(Acquisitions_copy.loc[cond1, "acquired_at"].max())

print("="*60)
print("CAD")
cond2 = (Acquisitions_copy['price_currency_code'] =="CAD")
print(Acquisitions_copy.loc[cond2, "acquired_at"].min())
print(Acquisitions_copy.loc[cond2, "acquired_at"].max())

print("="*60)
print("EUR")
cond3 = (Acquisitions_copy['price_currency_code'] =="EUR")
print(Acquisitions_copy.loc[cond3, "acquired_at"].min())
print(Acquisitions_copy.loc[cond3, "acquired_at"].max())

print("="*60)
print("GBP")
cond4 = (Acquisitions_copy['price_currency_code'] =="GBP")
print(Acquisitions_copy.loc[cond4, "acquired_at"].min())
print(Acquisitions_copy.loc[cond4, "acquired_at"].max())

print("="*60)
print("JPY")
cond5 = (Acquisitions_copy['price_currency_code'] =="JPY")
print(Acquisitions_copy.loc[cond5, "acquired_at"].min())
print(Acquisitions_copy.loc[cond5, "acquired_at"].max())

print("="*60)
print("SEK")
cond6 = (Acquisitions_copy['price_currency_code'] =="SEK")
print(Acquisitions_copy.loc[cond6, "acquired_at"].min())
print(Acquisitions_copy.loc[cond6, "acquired_at"].max())

AUD
2013-03-01 00:00:00
2013-03-01 00:00:00
CAD
2006-01-10 00:00:00
2012-09-13 00:00:00
EUR
1974-07-07 00:00:00
2013-11-05 00:00:00
GBP
1999-04-27 00:00:00
2013-11-27 00:00:00
JPY
1987-06-16 00:00:00
2013-06-18 00:00:00
SEK
2007-06-11 00:00:00
2013-10-23 00:00:00


In [29]:
# 환율 계산 함수

def cal_rate(code):
    if code=="AUD":
        cond1 = (usd_aud["observation_date"]>="2013-03-01") & (usd_aud["observation_date"]<="2013-03-01")
        result = usd_aud.loc[cond1, "DEXUSAL"].mean()

    if code=="CAD":
        cond2 = (usd_cad["observation_date"]>="2006-01-10") & (usd_cad["observation_date"]<="2012-09-13")
        result = usd_cad.loc[cond2, "DEXCAUS"].mean()

    if code=="EUR":
        cond3 = (usd_eur["observation_date"]>="1974-07-07") & (usd_eur["observation_date"]<="2013-11-05")
        result = usd_eur.loc[cond3, "DEXUSEU"].mean()
    

    if code=="GBP":
        cond4 = (usd_gbp["observation_date"]>="1999-04-27") & (usd_gbp["observation_date"]<="2013-11-27")
        result = usd_gbp.loc[cond4, "DEXUSUK"].mean()

    if code=="JPY":
        cond5 = (usd_jpy["observation_date"]>="1987-06-16") & (usd_jpy["observation_date"]<="2013-06-18")
        result = usd_jpy.loc[cond5, "DEXJPUS"].mean()

    if code=="SEK":
        cond6 = (usd_sek["observation_date"]>="2007-06-11") & (usd_sek["observation_date"]<="2013-10-23")
        result = usd_sek.loc[cond6, "DEXSDUS"].mean()
    
    return result

In [30]:
# acqusitions price_amount 환율 적용 

aud_mean = cal_rate("AUD")
cad_mean = cal_rate("CAD")
eur_mean = cal_rate("EUR")
gbp_mean = cal_rate("GBP")
jpy_mean = cal_rate("JPY")
sek_mean = cal_rate("SEK")
usd_mean = 1.0

Acquisitions_copy["price_amount_usd"] = np.nan
Acquisitions_copy["acqusition_currency_rate"] = np.nan

avg_rate = {
    "AUD": aud_mean,   # 예: 0.75  (U.S. $ to 1 AUD 이면 곱셈)
    "CAD": cad_mean,   # 예: 0.85  (U.S. $ to 1 CAD 이면 곱셈)
    "EUR": eur_mean,
    "GBP": gbp_mean,
    "JPY": jpy_mean,   # 예: 110.0 (JPY to 1 USD 이면 나눗셈)
    "SEK": sek_mean,
    "USD": 1.0         # 이미 USD인 경우 그대로 사용
}

for cur, r in avg_rate.items():
    cond = Acquisitions_copy["price_currency_code"] == cur
    
    # 방향에 따라 곱하거나 나누기
    if cur in ["AUD", "CAD", "EUR", "GBP"]:   # "U.S. $ to 1 통화" 타입
        Acquisitions_copy.loc[cond, "price_amount_usd"] = (
            Acquisitions_copy.loc[cond, "price_amount"] * r
        )
        Acquisitions_copy.loc[cond, "acqusition_currency_rate"] = r
    elif cur in ["JPY", "SEK"]:               # "통화 to 1 U.S. $" 타입
        Acquisitions_copy.loc[cond, "price_amount_usd"] = (
            Acquisitions_copy.loc[cond, "price_amount"] / r
        )
        Acquisitions_copy.loc[cond, "acqusition_currency_rate"] = r
    else:  # USD 등
        Acquisitions_copy.loc[cond, "price_amount_usd"] = Acquisitions_copy.loc[cond, "price_amount"]
        Acquisitions_copy.loc[cond, "acqusition_currency_rate"] = r


#예시 aud
display(Acquisitions_copy.loc[cond1, ["price_currency_code", "acqusition_currency_rate", "price_amount", "price_amount_usd"]])

,price_currency_code,acqusition_currency_rate,price_amount,price_amount_usd
9466,AUD,1.021,25000000.0,25525000.0


In [31]:
Acquisitions_copy

,acquisition_id,acquiring_c_id,acquired_c_id,term_code,price_amount,price_currency_code,acquired_at,is_acq_price_private,is_acquisitions_acq_at_missing,price_amount_usd,acqusition_currency_rate
0,1,c:11,c:10,NaN,2.000000e+07,USD,2007-05-30,0,0,2.000000e+07,1.0
1,7,c:59,c:72,cash,0.000000e+00,USD,2007-07-01,0,0,0.000000e+00,1.0
2,8,c:24,c:132,cash,2.800000e+08,USD,2007-05-01,0,0,2.800000e+08,1.0
3,9,c:59,c:155,cash,0.000000e+00,USD,2007-06-01,0,0,0.000000e+00,1.0
4,10,c:212,c:215,cash,2.500000e+07,USD,2007-07-01,0,0,2.500000e+07,1.0
...,...,...,...,...,...,...,...,...,...,...,...
9557,10525,c:267859,c:285995,stock,2.220000e+08,USD,2013-12-11,0,0,2.220000e+08,1.0
9558,10526,c:285493,c:286006,NaN,NaN,USD,2013-12-09,1,0,NaN,1.0
9559,10527,c:223792,c:220208,NaN,2.600000e+10,USD,2007-07-03,0,0,2.600000e+10,1.0
9560,10528,c:227620,c:286176,cash,5.240000e+07,USD,2013-12-12,0,0,5.240000e+07,1.0


In [ ]:
# # csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
# Acquisitions_copy.to_csv('clean_acquisitions_v2.csv', index=False)
# print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")

전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.


## Degrees 데이터 전처리

In [32]:
# 컬럼 제거
Degrees_copy = Degrees.copy()
Degrees_copy = Degrees_copy.drop(columns=['id', 'created_at', 'updated_at'])

In [33]:
# object_id 컬럼명 변경
alter_columns = Degrees_copy.rename(columns={
    'object_id': 'degrees_p_id'
}, inplace=True)

In [34]:
# degree 중복행 제거
Degrees_copy = Degrees_copy.drop_duplicates()

In [35]:
# degree_type 카테고리화 (FINAL v4.3: years + degree/graduated/diploma + institution(business/management) 보정)
import pandas as pd
import re

def _has_token(s: str, token: str) -> bool:
    return re.search(rf"(^|[^a-z]){re.escape(token)}([^a-z]|$)", s) is not None

def _norm(s: str) -> str:
    s = s.lower().strip()
    s = s.replace("’", "'")
    s = re.sub(r"[\/,;|]+", " ", s)
    s = re.sub(r"[\.\-]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

BACHELOR_MISSPELL = re.compile(r"\b(batchelor|bachelo?r|bachelro|bachleor|bachelar|bac?hlor|bachlor|bchelor|bachilor)\b")
MASTER_MISSPELL   = re.compile(r"\b(mastre|maste?r|maaster|mst?r|mster|mastes)\b")
DOCTOR_MISSPELL   = re.compile(r"\b(docter|docotor|doctoraet|doctorat|doctral|doc?toral)\b")

BACH_EXTRA_TOKENS = {
    "bdes","bbus","bacc","bmath","badmin","bsw","bkin","bmedsci",
    "bcomp","bcompsci","bcsci","bit","binfo","binf","bscit","bmis",
    "bstat","bbi","bcomm","bcommm","bhons","hons"
}
MAST_EXTRA_TOKENS = {
    "msca","msed","msw","macc","mfin","mib","mim","mpaff","mstat",
    "mcs","mcit","mism","msci","pgcert","pgdip","gradcert","graddip",
    "gcert","gdip","pgce","mres"
}
DOC_EXTRA_TOKENS = {"engd","dsc","scd","pharmd","habil","habilitation","drph"}

DATA_SCI_PAT = re.compile(r"\b(data\s*science|data\s*sci|datascience|data\-science)\b")
DOC_SCI_PAT  = re.compile(r"\b(doctor\s*of\s*science|d\s*sc|dsc|scd|dr\s*sc|drsc)\b")
DS_TOKEN_PAT = re.compile(r"(^|[^a-z])d\s*s([^a-z]|$)")

DEGREE_LIKE_PAT = re.compile(r"\b(degr(e{0,2})?e{0,2}|degeree|degre|degee)\b")
GRAD_LIKE_PAT   = re.compile(r"\b(grad(u)?at(e)?d|graduat(ed)?|graduted|graduatd|gradated|gradut(ed)?|graduation|graduashun)\b")
DIPLOMA_LIKE_PAT = re.compile(r"\b(diploma|diplome|diplom)\b")

# ✅ years 표기 전부 인식 (3years / 3-year / 3 yr / 3yrs)
YEARS_DEGREE_PAT = re.compile(r"\b(\d{1,2})\s*(?:\-?\s*)?(?:year|years|yr|yrs)\b")

DEGREE_ANY  = re.compile(r"degree")
DIPLOMA_ANY = re.compile(r"diplom")
GRAD_ANY    = re.compile(r"graduat|graduation|gradut|gradated|grad")

LAW_JD_PAT  = re.compile(r"\b(j\s*d|jd|j\.?\s*d\.?)\b")
LAW_LLM_PAT = re.compile(r"\b(l\s*l\s*m|llm|l\.?\s*l\.?\s*m\.?)\b")
LAW_LLB_PAT = re.compile(r"\b(l\s*l\s*b|llb|l\.?\s*l\.?\s*b\.?)\b")


def classify_degree(deg_type, institution):

    if pd.isna(deg_type) or (isinstance(deg_type, str) and deg_type.strip() == ""):
        return pd.NA

    s = _norm(str(deg_type))
    inst = _norm("" if pd.isna(institution) else str(institution))

    if s in {"nan", "none", "null", "n/a", "na", "unknown", "undefined"}:
        return pd.NA

    # (0) JD/LLM/LLB 단독 확정
    if LAW_JD_PAT.search(s):
        return "Doctorate"
    if LAW_LLM_PAT.search(s):
        return "Master’s degree"
    if LAW_LLB_PAT.search(s):
        return "Bachelor’s degree"

    # (0) N-years degree: "3years" 같은 '년수만 적힌 케이스'도 무조건 분류
    m_year = YEARS_DEGREE_PAT.search(s)
    if m_year:
        yrs = int(m_year.group(1))
        if yrs <= 2:
            return "Associate degree"
        if 3 <= yrs <= 4:
            return "Bachelor’s degree"
        if 5 <= yrs <= 6:
            return "Master’s degree"
        if yrs >= 7:
            return "Doctorate"

    # (1) DS / Data Science
    if DOC_SCI_PAT.search(s):
        return "Doctorate"

    if DATA_SCI_PAT.search(s):
        if ("phd" in s) or ("doctor" in s) or re.search(r"\bph\s*d\b", s) or _has_token(s, "dba") or _has_token(s, "edd"):
            return "Doctorate"
        if ("master" in s) or re.search(r"\bm\s*s\b", s) or re.search(r"\bm\s*a\b", s) or _has_token(s, "msc") or _has_token(s, "mba") or _has_token(s, "meng") or _has_token(s, "mph"):
            return "Master’s degree"
        if ("bachelor" in s) or re.search(r"\bb\s*s\b", s) or re.search(r"\bb\s*a\b", s) or _has_token(s, "bsc") or _has_token(s, "ba"):
            return "Bachelor’s degree"
        return "Others"

    if DS_TOKEN_PAT.search(s) or _has_token(s, "ds"):
        if ("phd" in s) or ("doctor" in s) or re.search(r"\bph\s*d\b", s) or _has_token(s, "dsc") or _has_token(s, "scd"):
            return "Doctorate"
        if ("master" in s) or re.search(r"\bm\s*s\b", s) or re.search(r"\bm\s*a\b", s) or _has_token(s, "msc") or _has_token(s, "mba") or _has_token(s, "meng") or _has_token(s, "mph"):
            return "Master’s degree"
        if ("bachelor" in s) or re.search(r"\bb\s*s\b", s) or re.search(r"\bb\s*a\b", s) or _has_token(s, "bsc") or _has_token(s, "ba"):
            return "Bachelor’s degree"
        return "Others"

    # (2) postgraduate/graduate diploma/certificate => Master
    if ("postgraduate" in s) or ("post graduate" in s) or ("postgraduate diploma" in s) or ("pg diploma" in s):
        return "Master’s degree"
    if ("graduate certificate" in s) or ("graduate diploma" in s) or _has_token(s, "gradcert") or _has_token(s, "graddip"):
        return "Master’s degree"
    if _has_token(s, "pgcert") or _has_token(s, "pgdip") or _has_token(s, "pgce"):
        return "Master’s degree"
    if _has_token(s, "dea") or _has_token(s, "dess"):
        return "Master’s degree"
    if _has_token(s, "diplom") or _has_token(s, "dipl"):
        return "Master’s degree"

    # (3) degree/graduated/graduation/diploma/law degree + institution 보정
    trigger_like = (
        DEGREE_LIKE_PAT.search(s)
        or GRAD_LIKE_PAT.search(s)
        or DIPLOMA_LIKE_PAT.search(s)
        or DEGREE_ANY.search(s)
        or DIPLOMA_ANY.search(s)
        or GRAD_ANY.search(s)
        or ("law degree" in s)
    )
    if trigger_like:
        # ✅ [ADD v4.3] business/management school 힌트 -> "석사"로 보정 (graduate/MBA 뉘앙스 있을 때만)
        inst_has_biz = (
            ("school of business" in inst)
            or ("business school" in inst)
            or ("school of management" in inst)
            or ("management school" in inst)
            or ("faculty of business" in inst)
            or ("business administration" in inst)
            or ("commerce" in inst)  # Rotman Commerce 같은 케이스 방어
        )
        inst_has_grad_hint = (
            ("graduate" in inst)
            or ("postgraduate" in inst)
            or ("post graduate" in inst)
            or ("masters" in inst)
            or ("master" in inst)
            or ("mba" in inst) or ("m b a" in inst)
            or ("macc" in inst) or ("mfin" in inst) or ("ms" in inst) or ("msc" in inst)
        )
        if inst_has_biz and inst_has_grad_hint:
            return "Master’s degree"

        # (기존) university/college 보정
        if ("university" in inst) or _has_token(inst, "univ"):
            return "Bachelor’s degree"
        if ("college" in inst):
            return "Associate degree"

    # (4) Associate
    if (
        "associate" in s
        or "community college" in s
        or "high school" in s
        or "ged" in s
        or re.search(r"\bhs\b", s)
        or "as" in s
        or "aa" in s
        or s in ["aas", "a a s", "a s", "a a", "ad", "diploma", "certificate",
                 "vocational", "technical", "tech school", "trade school",
                 "2 year degree", "two year degree", "sub degree",
                 "higher diploma", "hnd"]
        or _has_token(s, "nd") or _has_token(s, "ond")
    ):
        return "Associate degree"

    # (5) Bachelor
    if (
        "bachelor" in s
        or "undergrad" in s
        or re.search(r"\bb\s*a\b", s)
        or re.search(r"\bb\s*s\b", s)
        or re.search(r"\bb\s*sc\b", s)
        or "bsc" in s
        or "bcs" in s
        or "bfa" in s
        or "be" in s
        or "llb" in s
        or "basc" in s
        or "bba" in s
        or "btech" in s
        or "bcom" in s
        or "bbs" in s
        or "bid" in s
        or s in ["beng", "b eng", "bengineering", "baur", "barch", "bva", "bm",
                 "bmus", "bn", "bsn", "bph", "bba", "bcomm", "blaw", "bsw",
                 "b ed", "bed", "beng(hons)", "bth", "bpharm", "bsee",
                 "bsce", "bsme", "bmet", "bis", "bachlor", "bse"]
        or BACHELOR_MISSPELL.search(s) is not None
        or re.search(r"\ba\s*b\b", s) or _has_token(s, "ab")
        or re.search(r"\bs\s*b\b", s) or _has_token(s, "sb")
        or re.search(r"\bsc\s*b\b", s) or _has_token(s, "scb")
        or "licenciatura" in s or _has_token(s, "licence") or "laurea" in s or "grado" in s
        or any(_has_token(s, t) for t in BACH_EXTRA_TOKENS)
        or ("honours" in s or "honors" in s or _has_token(s, "hons"))
    ):
        return "Bachelor’s degree"

    # (6) Master
    if (
        "master" in s
        or "graduate school" in s
        or "grad school" in s
        or "mba" in s
        or re.search(r"\bm\s*s\b", s)
        or re.search(r"\bm\s*a\b", s)
        or "msc" in s
        or "msba" in s
        or "mphil" in s
        or "llm" in s
        or "mpa" in s
        or "mse" in s
        or "meng" in s
        or s in ["mfa", "mim", "mlis", "mm", "mn", "msn", "march", "m div",
                 "mha", "mpp", "msw", "macc", "mfin", "m ed", "mcm", "mlt",
                 "a m", "pg"]
        or MASTER_MISSPELL.search(s) is not None
        or re.search(r"\bs\s*m\b", s) or _has_token(s, "sm")
        or re.search(r"\bsc\s*m\b", s) or _has_token(s, "scm")
        or re.search(r"\bm\s*eng\b", s) or _has_token(s, "meng")
        or _has_token(s, "mph") or _has_token(s, "mps") or _has_token(s, "me")
        or _has_token(s, "mres") or _has_token(s, "msci")
        or any(_has_token(s, t) for t in MAST_EXTRA_TOKENS)
    ):
        return "Master’s degree"

    # (7) Doctorate
    if (
        "phd" in s
        or re.search(r"\bph\s*d\b", s)
        or "doctorate" in s
        or "doctoral" in s
        or "doctor of" in s
        or s in ["md", "jd", "edd", "dphil"]
        or s in ["dba", "psy d", "psyd", "dpt", "dvm", "aud", "dc", "do",
                 "drph", "scd", "thd", "j s d", "s j d", "ll d", "abd", "j d"]
        or DOCTOR_MISSPELL.search(s) is not None
        or _has_token(s, "dds") or _has_token(s, "dmd") or _has_token(s, "drph") or _has_token(s, "scd")
        or any(_has_token(s, t) for t in DOC_EXTRA_TOKENS)
        or "dr rer nat" in s or "dr rer" in s or "rer nat" in s
    ):
        return "Doctorate"

    # Rescue Pass
    if (
        ("doctor" in s) or ("phd" in s) or re.search(r"\bph\s*d\b", s)
        or _has_token(s, "dba") or _has_token(s, "edd") or _has_token(s, "dphil")
        or any(_has_token(s, t) for t in DOC_EXTRA_TOKENS) or "dr rer nat" in s or "habilitation" in s
    ):
        return "Doctorate"

    if (
        ("master" in s) or _has_token(s, "mba")
        or re.search(r"\bm\s*s\b", s) or re.search(r"\bm\s*a\b", s)
        or _has_token(s, "msc") or _has_token(s, "meng") or _has_token(s, "mph")
        or _has_token(s, "mpp") or _has_token(s, "mpa") or _has_token(s, "llm")
        or _has_token(s, "sm") or _has_token(s, "scm") or _has_token(s, "me") or _has_token(s, "mps") or _has_token(s, "mres")
        or any(_has_token(s, t) for t in MAST_EXTRA_TOKENS)
        or ("postgraduate" in s) or ("post graduate" in s) or ("graduate certificate" in s) or ("graduate diploma" in s)
    ):
        return "Master’s degree"

    if (
        ("bachelor" in s)
        or re.search(r"\bb\s*a\b", s) or re.search(r"\bb\s*s\b", s) or re.search(r"\bb\s*sc\b", s)
        or BACHELOR_MISSPELL.search(s)
        or _has_token(s, "ab") or _has_token(s, "sb") or _has_token(s, "scb")
        or any(_has_token(s, t) for t in BACH_EXTRA_TOKENS)
        or ("honours" in s or "honors" in s)
    ):
        return "Bachelor’s degree"

    if ("associate" in s) or _has_token(s, "aas") or _has_token(s, "hnd") or ("community college" in s):
        return "Associate degree"

    return "Others"


Degrees_copy["cat_degrees_degree_type"] = Degrees_copy.apply(
    lambda r: classify_degree(r["degree_type"], r["institution"]),
    axis=1
)

category_order = ["Associate degree", "Bachelor’s degree", "Master’s degree", "Doctorate", "Others"]
Degrees_copy["cat_degrees_degree_type"] = pd.Categorical(
    Degrees_copy["cat_degrees_degree_type"],
    categories=category_order,
    ordered=True
)


In [36]:
# degree_type 카테고리화 (FINAL v4.4: custom tokens + business administration BA + med/mce/pgdm/hba MS + bca/dec/bsfs)
import pandas as pd
import re

def _has_token(s: str, token: str) -> bool:
    return re.search(rf"(^|[^a-z]){re.escape(token)}([^a-z]|$)", s) is not None

def _norm(s: str) -> str:
    s = s.lower().strip()
    s = s.replace("’", "'")
    s = re.sub(r"[\/,;|]+", " ", s)
    s = re.sub(r"[\.\-]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

BACHELOR_MISSPELL = re.compile(r"\b(batchelor|bachelo?r|bachelro|bachleor|bachelar|bac?hlor|bachlor|bchelor|bachilor)\b")
MASTER_MISSPELL   = re.compile(r"\b(mastre|maste?r|maaster|mst?r|mster|mastes)\b")
DOCTOR_MISSPELL   = re.compile(r"\b(docter|docotor|doctoraet|doctorat|doctral|doc?toral)\b")

BACH_EXTRA_TOKENS = {
    "bdes","bbus","bacc","bmath","badmin","bsw","bkin","bmedsci",
    "bcomp","bcompsci","bcsci","bit","binfo","binf","bscit","bmis",
    "bstat","bbi","bcomm","bcommm","bhons","hons",
    # ✅ [ADD v4.4] 요청 추가
    "bca","bsfs"
}
MAST_EXTRA_TOKENS = {
    "msca","msed","msw","macc","mfin","mib","mim","mpaff","mstat",
    "mcs","mcit","mism","msci","pgcert","pgdip","gradcert","graddip",
    "gcert","gdip","pgce","mres",
    # ✅ [ADD v4.4] 요청 추가
    "pgdm","hba","mce"
}
DOC_EXTRA_TOKENS = {"engd","dsc","scd","pharmd","habil","habilitation","drph"}

DATA_SCI_PAT = re.compile(r"\b(data\s*science|data\s*sci|datascience|data\-science)\b")
DOC_SCI_PAT  = re.compile(r"\b(doctor\s*of\s*science|d\s*sc|dsc|scd|dr\s*sc|drsc)\b")
DS_TOKEN_PAT = re.compile(r"(^|[^a-z])d\s*s([^a-z]|$)")

DEGREE_LIKE_PAT = re.compile(r"\b(degr(e{0,2})?e{0,2}|degeree|degre|degee)\b")
GRAD_LIKE_PAT   = re.compile(r"\b(grad(u)?at(e)?d|graduat(ed)?|graduted|graduatd|gradated|gradut(ed)?|graduation|graduashun)\b")
DIPLOMA_LIKE_PAT = re.compile(r"\b(diploma|diplome|diplom)\b")

YEARS_DEGREE_PAT = re.compile(r"\b(\d{1,2})\s*(?:\-?\s*)?(?:year|years|yr|yrs)\b")

DEGREE_ANY  = re.compile(r"degree")
DIPLOMA_ANY = re.compile(r"diplom")
GRAD_ANY    = re.compile(r"graduat|graduation|gradut|gradated|grad")

LAW_JD_PAT  = re.compile(r"\b(j\s*d|jd|j\.?\s*d\.?)\b")
LAW_LLM_PAT = re.compile(r"\b(l\s*l\s*m|llm|l\.?\s*l\.?\s*m\.?)\b")
LAW_LLB_PAT = re.compile(r"\b(l\s*l\s*b|llb|l\.?\s*l\.?\s*b\.?)\b")

# ✅ [ADD v4.4] 요청한 “단독 토큰/문구”를 강제 분류하는 룰 (토큰만 들어가도 잡힘)
CUSTOM_ASSOC_TOKENS = {"dec"}                 # 전문학사
CUSTOM_BACH_TOKENS  = {"bca", "bsfs"}         # 학사
CUSTOM_MAST_TOKENS  = {"mce", "pgdm", "hba"}  # 석사
# ✅ [ADD v4.4] "med"는 기존에 석사로 쓰고 있었으니, 토큰으로도 확정
CUSTOM_MAST_TOKENS |= {"med"}

def classify_degree(deg_type, institution):

    if pd.isna(deg_type) or (isinstance(deg_type, str) and deg_type.strip() == ""):
        return pd.NA

    s = _norm(str(deg_type))
    inst = _norm("" if pd.isna(institution) else str(institution))

    if s in {"nan", "none", "null", "n/a", "na", "unknown", "undefined"}:
        return pd.NA

    # ✅ [ADD v4.4] (0) 커스텀 토큰 “최우선” 확정 (다른 규칙보다 먼저)
    # - 토큰만 들어가도 확정되게 _has_token 사용
    if any(_has_token(s, t) for t in CUSTOM_ASSOC_TOKENS):
        return "Associate degree"
    if any(_has_token(s, t) for t in CUSTOM_MAST_TOKENS):
        return "Master’s degree"
    if any(_has_token(s, t) for t in CUSTOM_BACH_TOKENS):
        return "Bachelor’s degree"

    # ✅ [ADD v4.4] "business administration"은 학사로 확정 (요청)
    # - 전공명+degree 형태로 와도 포함되면 학사로 처리
    if ("business administration" in s) or ("business administraion" in s):  # 오탈자까지
        return "Bachelor’s degree"

    # (0) JD/LLM/LLB 단독 확정
    if LAW_JD_PAT.search(s):
        return "Doctorate"
    if LAW_LLM_PAT.search(s):
        return "Master’s degree"
    if LAW_LLB_PAT.search(s):
        return "Bachelor’s degree"

    # (0) N-years degree: "3years" 같은 '년수만 적힌 케이스'도 무조건 분류
    m_year = YEARS_DEGREE_PAT.search(s)
    if m_year:
        yrs = int(m_year.group(1))
        if yrs <= 2:
            return "Associate degree"
        if 3 <= yrs <= 4:
            return "Bachelor’s degree"
        if 5 <= yrs <= 6:
            return "Master’s degree"
        if yrs >= 7:
            return "Doctorate"

    # (1) DS / Data Science
    if DOC_SCI_PAT.search(s):
        return "Doctorate"

    if DATA_SCI_PAT.search(s):
        if ("phd" in s) or ("doctor" in s) or re.search(r"\bph\s*d\b", s) or _has_token(s, "dba") or _has_token(s, "edd"):
            return "Doctorate"
        if ("master" in s) or re.search(r"\bm\s*s\b", s) or re.search(r"\bm\s*a\b", s) or _has_token(s, "msc") or _has_token(s, "mba") or _has_token(s, "meng") or _has_token(s, "mph"):
            return "Master’s degree"
        if ("bachelor" in s) or re.search(r"\bb\s*s\b", s) or re.search(r"\bb\s*a\b", s) or _has_token(s, "bsc") or _has_token(s, "ba"):
            return "Bachelor’s degree"
        return "Others"

    if DS_TOKEN_PAT.search(s) or _has_token(s, "ds"):
        if ("phd" in s) or ("doctor" in s) or re.search(r"\bph\s*d\b", s) or _has_token(s, "dsc") or _has_token(s, "scd"):
            return "Doctorate"
        if ("master" in s) or re.search(r"\bm\s*s\b", s) or re.search(r"\bm\s*a\b", s) or _has_token(s, "msc") or _has_token(s, "mba") or _has_token(s, "meng") or _has_token(s, "mph"):
            return "Master’s degree"
        if ("bachelor" in s) or re.search(r"\bb\s*s\b", s) or re.search(r"\bb\s*a\b", s) or _has_token(s, "bsc") or _has_token(s, "ba"):
            return "Bachelor’s degree"
        return "Others"

    # (2) postgraduate/graduate diploma/certificate => Master
    if ("postgraduate" in s) or ("post graduate" in s) or ("postgraduate diploma" in s) or ("pg diploma" in s):
        return "Master’s degree"
    if ("graduate certificate" in s) or ("graduate diploma" in s) or _has_token(s, "gradcert") or _has_token(s, "graddip"):
        return "Master’s degree"
    if _has_token(s, "pgcert") or _has_token(s, "pgdip") or _has_token(s, "pgce"):
        return "Master’s degree"
    if _has_token(s, "dea") or _has_token(s, "dess"):
        return "Master’s degree"
    if _has_token(s, "diplom") or _has_token(s, "dipl"):
        return "Master’s degree"

    # (3) degree/graduated/graduation/diploma/law degree + institution 보정
    trigger_like = (
        DEGREE_LIKE_PAT.search(s)
        or GRAD_LIKE_PAT.search(s)
        or DIPLOMA_LIKE_PAT.search(s)
        or DEGREE_ANY.search(s)
        or DIPLOMA_ANY.search(s)
        or GRAD_ANY.search(s)
        or ("law degree" in s)
    )
    if trigger_like:
        # business/management school 힌트 -> "석사"로 보정 (graduate/MBA 뉘앙스 있을 때만)
        inst_has_biz = (
            ("school of business" in inst)
            or ("business school" in inst)
            or ("school of management" in inst)
            or ("management school" in inst)
            or ("faculty of business" in inst)
            or ("business administration" in inst)
            or ("commerce" in inst)
        )
        inst_has_grad_hint = (
            ("graduate" in inst)
            or ("postgraduate" in inst)
            or ("post graduate" in inst)
            or ("masters" in inst)
            or ("master" in inst)
            or ("mba" in inst) or ("m b a" in inst)
            or ("macc" in inst) or ("mfin" in inst) or ("ms" in inst) or ("msc" in inst)
        )
        if inst_has_biz and inst_has_grad_hint:
            return "Master’s degree"

        if ("university" in inst) or _has_token(inst, "univ"):
            return "Bachelor’s degree"
        if ("college" in inst):
            return "Associate degree"

    # (4) Associate
    if (
        "associate" in s
        or "community college" in s
        or "high school" in s
        or "ged" in s
        or re.search(r"\bhs\b", s)
        or "as" in s
        or "aa" in s
        or s in ["aas", "a a s", "a s", "a a", "ad", "diploma", "certificate",
                 "vocational", "technical", "tech school", "trade school",
                 "2 year degree", "two year degree", "sub degree",
                 "higher diploma", "hnd"]
        or _has_token(s, "nd") or _has_token(s, "ond")
    ):
        return "Associate degree"

    # (5) Bachelor
    if (
        "bachelor" in s
        or "undergrad" in s
        or re.search(r"\bb\s*a\b", s)
        or re.search(r"\bb\s*s\b", s)
        or re.search(r"\bb\s*sc\b", s)
        or "bsc" in s
        or "bcs" in s
        or "bfa" in s
        or "be" in s
        or "llb" in s
        or "basc" in s
        or "bba" in s
        or "btech" in s
        or "bcom" in s
        or "bbs" in s
        or "bid" in s
        or s in ["beng", "b eng", "bengineering", "baur", "barch", "bva", "bm",
                 "bmus", "bn", "bsn", "bph", "bba", "bcomm", "blaw", "bsw",
                 "b ed", "bed", "beng(hons)", "bth", "bpharm", "bsee",
                 "bsce", "bsme", "bmet", "bis", "bachlor", "bse"]
        or BACHELOR_MISSPELL.search(s) is not None
        or re.search(r"\ba\s*b\b", s) or _has_token(s, "ab")
        or re.search(r"\bs\s*b\b", s) or _has_token(s, "sb")
        or re.search(r"\bsc\s*b\b", s) or _has_token(s, "scb")
        or "licenciatura" in s or _has_token(s, "licence") or "laurea" in s or "grado" in s
        or any(_has_token(s, t) for t in BACH_EXTRA_TOKENS)
        or ("honours" in s or "honors" in s or _has_token(s, "hons"))
    ):
        return "Bachelor’s degree"

    # (6) Master
    if (
        "master" in s
        or "graduate school" in s
        or "grad school" in s
        or "mba" in s
        or re.search(r"\bm\s*s\b", s)
        or re.search(r"\bm\s*a\b", s)
        or "msc" in s
        or "msba" in s
        or "mphil" in s
        or "llm" in s
        or "mpa" in s
        or "mse" in s
        or "meng" in s
        or s in ["mfa", "mim", "mlis", "mm", "mn", "msn", "march", "m div",
                 "mha", "mpp", "msw", "macc", "mfin", "m ed", "mcm", "mlt",
                 "a m", "pg"]
        or MASTER_MISSPELL.search(s) is not None
        or re.search(r"\bs\s*m\b", s) or _has_token(s, "sm")
        or re.search(r"\bsc\s*m\b", s) or _has_token(s, "scm")
        or re.search(r"\bm\s*eng\b", s) or _has_token(s, "meng")
        or _has_token(s, "mph") or _has_token(s, "mps") or _has_token(s, "me")
        or _has_token(s, "mres") or _has_token(s, "msci")
        or any(_has_token(s, t) for t in MAST_EXTRA_TOKENS)
    ):
        return "Master’s degree"

    # (7) Doctorate
    if (
        "phd" in s
        or re.search(r"\bph\s*d\b", s)
        or "doctorate" in s
        or "doctoral" in s
        or "doctor of" in s
        or s in ["md", "jd", "edd", "dphil"]
        or s in ["dba", "psy d", "psyd", "dpt", "dvm", "aud", "dc", "do",
                 "drph", "scd", "thd", "j s d", "s j d", "ll d", "abd", "j d"]
        or DOCTOR_MISSPELL.search(s) is not None
        or _has_token(s, "dds") or _has_token(s, "dmd") or _has_token(s, "drph") or _has_token(s, "scd")
        or any(_has_token(s, t) for t in DOC_EXTRA_TOKENS)
        or "dr rer nat" in s or "dr rer" in s or "rer nat" in s
    ):
        return "Doctorate"

    # Rescue Pass
    if (
        ("doctor" in s) or ("phd" in s) or re.search(r"\bph\s*d\b", s)
        or _has_token(s, "dba") or _has_token(s, "edd") or _has_token(s, "dphil")
        or any(_has_token(s, t) for t in DOC_EXTRA_TOKENS) or "dr rer nat" in s or "habilitation" in s
    ):
        return "Doctorate"

    if (
        ("master" in s) or _has_token(s, "mba")
        or re.search(r"\bm\s*s\b", s) or re.search(r"\bm\s*a\b", s)
        or _has_token(s, "msc") or _has_token(s, "meng") or _has_token(s, "mph")
        or _has_token(s, "mpp") or _has_token(s, "mpa") or _has_token(s, "llm")
        or _has_token(s, "sm") or _has_token(s, "scm") or _has_token(s, "me") or _has_token(s, "mps") or _has_token(s, "mres")
        or any(_has_token(s, t) for t in MAST_EXTRA_TOKENS)
        or ("postgraduate" in s) or ("post graduate" in s) or ("graduate certificate" in s) or ("graduate diploma" in s)
    ):
        return "Master’s degree"

    if (
        ("bachelor" in s)
        or re.search(r"\bb\s*a\b", s) or re.search(r"\bb\s*s\b", s) or re.search(r"\bb\s*sc\b", s)
        or BACHELOR_MISSPELL.search(s)
        or _has_token(s, "ab") or _has_token(s, "sb") or _has_token(s, "scb")
        or any(_has_token(s, t) for t in BACH_EXTRA_TOKENS)
        or ("honours" in s or "honors" in s)
    ):
        return "Bachelor’s degree"

    if ("associate" in s) or _has_token(s, "aas") or _has_token(s, "hnd") or ("community college" in s):
        return "Associate degree"

    return "Others"


Degrees_copy["cat_degrees_degree_type"] = Degrees_copy.apply(
    lambda r: classify_degree(r["degree_type"], r["institution"]),
    axis=1
)

category_order = ["Associate degree", "Bachelor’s degree", "Master’s degree", "Doctorate", "Others"]
Degrees_copy["cat_degrees_degree_type"] = pd.Categorical(
    Degrees_copy["cat_degrees_degree_type"],
    categories=category_order,
    ordered=True
)


In [37]:
# 매핑 딕셔너리 정의
degree_mapping = {
    "Associate degree": 1,
    "Bachelor’s degree": 2,
    "Master’s degree": 3,
    "Doctorate": 4,
    "Others": 0
}

# 'cat_degrees_degree_type' 컬럼의 값을 매핑
Degrees_copy["degree_level"] = Degrees_copy["cat_degrees_degree_type"].map(degree_mapping)

# degree 결측값 플래그 컬럼 생성
Degrees_copy["is_degree_missing"] = Degrees_copy["degree_type"].isna().astype(int)


In [38]:
Degrees_copy

,degrees_p_id,degree_type,subject,institution,graduated_at,cat_degrees_degree_type,degree_level,is_degree_missing
0,p:6117,MBA,NaN,NaN,NaN,Master’s degree,3,0
1,p:6136,BA,"English, French","Washington University, St. Louis",1990-01-01,Bachelor’s degree,2,0
2,p:6136,MS,Mass Communication,Boston University,1992-01-01,Master’s degree,3,0
3,p:6005,MS,Internet Technology,University of Greenwich,2006-01-01,Master’s degree,3,0
4,p:5832,BCS,"Computer Science, Psychology",Rice University,NaN,Bachelor’s degree,2,0
...,...,...,...,...,...,...,...,...
109605,p:268589,CPA,NaN,American Institute of Certtified Public Accoun...,NaN,Others,0,0
109606,p:268527,MS & BS,Engineering,South China University of Technology,NaN,Bachelor’s degree,2,0
109607,p:268527,PhD,Engineering,Clarkson University,NaN,Doctorate,4,0
109608,p:268528,B.S.,Electrical Engineering,Colorado State University,NaN,Bachelor’s degree,2,0


In [39]:

# subject 카테고리화

def is_missing_subject(x) -> bool:
    """subject가 결측으로 봐야 하는 값인지 판별"""
    if pd.isna(x):
        return True
    s = str(x).strip().lower()
    return s in {"", "nan", "none", "null", "missing value", "unknown"}

def tokenize_subject(x: str) -> list[str]:
    """
    "Computer Science, Psychology" / "International Business & Marketing"
    같은 복합 표기를 토큰화해서 키워드 매칭이 더 잘 되게 함
    """
    s = str(x).lower()
    # 구분자 통일 (콤마, &, /, and 등)
    s = re.sub(r"[/&]", ",", s)
    s = re.sub(r"\band\b", ",", s)
    parts = [p.strip() for p in s.split(",") if p.strip()]
    return parts if parts else [s.strip()]

def has_word(s: str, word: str) -> bool:
    return re.search(rf"\b{re.escape(word)}\b", s) is not None

DEGREE_LABEL_PAT = re.compile(
   r"""^(
        degree|graduate|undergraduate|postgraduate|
        high\ school|secondary|diploma|certificate|
        bachelor|bachelors\ degree|masters|master|master\ of\ science|doctorate|phd|ph\.d|
        honors|magna\ cum\ laude|cum\ laude|summa\ cum\ laude| 
        mba|m\.b\.a|ms|m\.s|msc|m\.sc|meng|m\.eng|
        ba|b\.a|bs|b\.s|bsc|b\.sc|be|b\.e|
        jd|j\.d|llb|ll\.b
    )$""",
    re.IGNORECASE | re.VERBOSE
)
# 포괄적인 라벨 -> genral studies
GENERIC_MAJOR_PAT = re.compile(
    r"""^(
        science|sciences|arts|humanities|engineering|
        business|commerce|management|
        it|cs|cis|eecs|ece|ee
    )$""",
    re.IGNORECASE | re.VERBOSE
)

ABBR_CS = {"it", "cs", "cis", "mis", "ict"}         # CS/IT계열
ABBR_EE = {"ee", "ece", "eecs"}                    # 전기/전자
ABBR_FIN = {"cpa"}                                 # 회계
ABBR_LAW = {"jd", "llb"}       
ENG_ABBR = {"ece", "eecs", "ee", "ict"}

REPLACE_MAP = {
    "managment": "management",
    "inernational": "international",
    "sevice": "service",
    "psycology": "psychology",
    "bachelor\x92s degree": "bachelors degree",
}

def normalize_text(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"[\.]", "", s)
    s = re.sub(r"[_\-]+", " ", s)
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    for k, v in REPLACE_MAP.items():
        s = s.replace(k, v)
    return s

def map_subject_20(x):
    # 0) 결측은 "Others"로 보내지 말고 결측 유지
    if is_missing_subject(x):
        return pd.NA

    tokens = tokenize_subject(x)
    s_all = " ".join(tokens)
    s_all = s_all.lower()
    s_all = re.sub(r"[_\-]+", " ", s_all)
    s_all = re.sub(r"[\.]", "", s_all)           
    s_all = re.sub(r"\s+", " ", s_all).strip()
    
    if DEGREE_LABEL_PAT.match(s_all):             
        return pd.NA 
    
    if "managment" in s_all:                     
        s_all = s_all.replace("managment", "management")
    
    if any(k in s_all for k in [
        "social studies", "social study",
        "social work", "human services", "social welfare", "social care"    
    ]):
        return "Psychology / Sociology"
    
    # 12) Marketing / Communications (Mass Communication 포함)
    if any(k in s_all for k in [
        "marketing", "communication", "communications", "mass communication",
        "media studies", "public relations", "pr", "journalism", "advertising",
        "branding", "digital media", "commerce", "media", "communications studies", "broadcasting", 
        "multimedia", "interactive media", "new media", "film studies", "media arts", "technology commercialization"
    ]) or has_word(s_all, "pr"):
        return "Marketing / Communications"

    # 1) Computer Science / Software (Internet Technology/IT 포함)
    if any(k in s_all for k in [
        "computer science", "software", "information systems", "information system",
        "computing", "programming", "internet", "information technology", "it ", "technology", "cybersecurity", "security",
        "cyber", "network", "networks", "web", "systems engineering", "informatics", "computer engineering", "computer engineer",
        "cloud", "devops", "database", "db", "security", "ee", "it", "systems", "management information systems", "mis", "computers", "computer", "cs",
        "cis", "symbolic systems", "information science", "computer applications", "computer application", "computer technology", "computer systems",
        "systems analysis", "computer and information science"
    ]) or has_word(s_all, "it") or has_word(s_all, "cs") or has_word(s_all, "ict") or has_word(s_all, "cse") or has_word(s_all, "cse") or has_word(s_all, "mca") or has_word(s_all, "ee cs"):
        return "Computer Science / Software"

    # 2) AI / Data / Robotics
    if any(k in s_all for k in [
        "artificial intelligence", "ai ", "machine learning", "deep learning",
        "data science", "data analytics", "statistics", "statistical",
        "robotics", "automation", "statistics", "statistical", "applied statistics",            
        "mathematics", "applied mathematics", "math", "operations research", "optimization",
        "quantitative", "analytics", "business analytics", "operational research"
    ]):
        return "AI / Data / Robotics"

    # 3) Electrical / Electronic Engineering
    if any(k in s_all for k in [
        "electrical engineering", "electronic", "electronics",
        "telecommunications", "signal processing", "embedded",
        "computer engineering", "microelectronics", "semiconductor",  
        "control engineering", "control systems", "iot", "internet of things", "microelectronics", "semiconductor", "chip", "circuits",
        "electrical"
    ]) or has_word(s_all, "ece") or has_word(s_all, "eecs") or has_word(s_all, "ee"):
        return "Electrical / Electronic Engineering"

    # 4) Mechanical / Industrial Engineering
    if any(k in s_all for k in [
        "mechanical engineering", "industrial engineering",
        "manufacturing", "mechatronics", "systems engineering",
        "engineering", "applied science", "applied sciences",  
        "civil engineering", "construction", "structural",      
        "chemical engineering", "materials", "material science",
        "aerospace", "automotive", "industrial design",         
        "engineering science", "applied engineering", "metallurgy", "applied sciences", "engineering and applied science", 
        "mechanical", "aeronautics", "aeronautics and astronautics", "industrial technology", "engineer"
    ]):
        return "Mechanical / Industrial Engineering"

    # 5) Physics / Chemistry
    if any(k in s_all for k in [
        "physics", "chemistry", "chemical", "physical chemistry", "geology", "geoscience", "earth science", "environmental science",
        "geography", "science", "kinesiology", "astronomy", "physical sciences", "natural sciences", "environmental studies"
    ]):
        return "Physics / Chemistry"

    # 6) Biology / Life Science
    if any(k in s_all for k in [
        "biology", "biochemistry", "molecular biology", "genetics",
        "life science", "biological", "microbiology", "bioinformatics", "biostatistics", "bs","physiology", "zoology",
        "agriculture", "nutrition", "exercise physiology"
    ]):
        return "Biology / Life Science"

    # 7) Medicine / Clinical Science
    if any(k in s_all for k in [
        "medicine", "medical", "clinical", "surgery", "dentistry", "nursing", "public health", "epidemiology", "healthcare", "health sciences" 
    ]):
        return "Medicine / Clinical Science"

    # 8) Biotechnology / Pharmaceutical
    if any(k in s_all for k in [
        "biotechnology", "pharmacology", "pharmacy", "drug", "pharmaceutical",
        "biomed", "biomedical", "biomed", "biomedical", "biomedical engineering",  
        "neuroscience", "immunology"
    ]):
        return "Biotechnology / Pharmaceutical"

    # 10) Finance / Accounting
    if any(k in s_all for k in [
        "finance", "accounting", "audit", "tax", "financial", "investment", "banking", "accountancy", "management", "public accountant",
        "accounts", "chartered accountant" 
    ]):
        return "Finance / Accounting"

    # 11) Economics
    if any(k in s_all for k in [
        "economics", "econometric", "economic", "econometrics", "economy", "political economy"
    ]):
        return "Economics"

    # 9) Business Administration (Finance/Econ보다 뒤에 두면 안 됨 → 위에서 finance/econ 먼저 잡음)
    if any(k in s_all for k in [
        "business administration", "mba", "management", "strategy",
        "entrepreneurship", "international business", "business", "business analytics", 
        "product management", "operations management", "organizational behavior", "organizational leadership", "leadership",
        "real estate", "international trade",                    
        "human resources", "hr"  
    ]):
        return "Business Administration"

    # 13) Psychology / Sociology
    if any(k in s_all for k in [
        "psychology", "sociology", "anthropology", "behavioral", "cognitive science", "american studies", "criminology"    
    ]):
        return "Psychology / Sociology"

    # 14) Political Science / International Relations
    if any(k in s_all for k in [
        "political science", "international relations", "politics", "diplomacy", "international studies", "global studies", "international affairs",
        "inernational trade", "foreign sevice", "international development", "industrial and labor relations", "ppe", "international political economy", "foreign service"
    ]):
        return "Political Science / International Relations"

    # 15) Law
    if any(k in s_all for k in [
        "law", "legal", "juris", "llb", "jd", "law and", "legal studies"
    ]):
        return "Law"

    # 16) Public Policy / Administration
    if any(k in s_all for k in [
        "public policy", "public administration", "government", "policy", "public affairs", "administration", "criminal justice"
    ]):
        return "Public Policy / Administration"

    # 17) Arts / Humanities  (언어/어학 여기로 보내기!)
    if any(k in s_all for k in [
        "philosophy", "history", "literature", "humanities", "arts", "fine art",
        "language", "languages", "english", "french", "spanish", "german",
        "linguistics", "translation", "interpretation", "religion", "theology", "classics", "music", "photography", "ba",
        "film", "art", "east asian studies", "asian studies", "european studies", "japanese", "russian studies",         
        "theatre", "theater", "acting",              
        "religious studies",                       
        "archaeology", "creative writing", "rhetoric", "music composition", "painting", "illustration"
    ]):
        return "Arts / Humanities"

    # 18) Design / Architecture
    if any(k in s_all for k in [
        "design", "ux", "ui", "architecture", "architect", "graphic design", "interaction design", "urban planning"
    ]):
        return "Design / Architecture"

    # 19) Education
    if any(k in s_all for k in [
        "education", "teaching", "pedagogy", "instructional", "educational", "curriculum"
    ]):
        return "Education"
    
    # 자유전공
    if any(k in s_all for k in [
        "general", "undeclared", "undecided", "general studies", "liberal arts", "liberal studies"
        "interdisciplinary", "open major", "self designed", "no major", "general study", "interdisciplinary studies"
    
    ]):
        return "General Studies"  

    # 20) Others / Unknown
    return "Others / Unknown"


# =========================
# 적용 + pd.Categorical
# =========================

Degrees_copy["cat_degrees_subject"] = Degrees_copy["subject"].apply(map_subject_20)

category_order_20 = [
    "Computer Science / Software",
    "AI / Data / Robotics",
    "Electrical / Electronic Engineering",
    "Mechanical / Industrial Engineering",
    "Physics / Chemistry",
    "Biology / Life Science",
    "Medicine / Clinical Science",
    "Biotechnology / Pharmaceutical",
    "Business Administration",
    "Finance / Accounting",
    "Economics",
    "Marketing / Communications",
    "Psychology / Sociology",
    "Political Science / International Relations",
    "Law",
    "Public Policy / Administration",
    "Arts / Humanities",
    "Design / Architecture",
    "Education",
    "General Studies",
    "Others / Unknown"
]

# 결측(pd.NA)은 그대로 남기고, 나머지만 Categorical로
Degrees_copy["cat_degrees_subject"] = pd.Categorical(
    Degrees_copy["cat_degrees_subject"],
    categories=category_order_20,
    ordered=True
)


In [40]:
Degrees_copy[Degrees_copy['cat_degrees_subject'] == 'Others / Unknown'][['subject', 'cat_degrees_subject']].value_counts()

subject                         cat_degrees_subject
Liberal Studies                 Others / Unknown       15
MD                              Others / Unknown       13
Psycology                       Others / Unknown       13
Industrial and Labor Relations  Others / Unknown       11
Telecom                         Others / Unknown       10
                                                       ..
Gastroenterology                Others / Unknown        1
Games                           Others / Unknown        1
Gallatin Division               Others / Unknown        1
GSP11                           Others / Unknown        1
www.givemedelivery.com          Others / Unknown        1
Name: count, Length: 1640, dtype: int64

In [41]:
# institution (학교명) 정규화

import unicodedata
import numpy as np 

def clean_basic(name: str) -> str:
    """기본 텍스트 정리 + 소문자 변환"""
    s = str(name).strip().lower()
    # 유니코드 정규화 (악센트 등 처리용)
    s = unicodedata.normalize("NFKC", s)
    # 이상한 따옴표 통일
    s = s.replace("\x92", "'").replace("’", "'").replace("`", "'")
    # 끝에 붙은 콤마/마침표 제거
    s = re.sub(r"[.,]\s*$", "", s)
    # 여러 공백 하나로
    s = re.sub(r"\s+", " ", s)
    return s

def extract_university_like(s: str) -> str | None:
    """
    university / college 가 들어간 핵심 기관명 추출
    예)
      'university of toronto school of law' → 'university of toronto'
      'boston university, school of management' → 'boston university'
      'babson college of business' → 'babson college'
    """
    # 1) 'university of ...' 패턴
    if "university of " in s:
        m = re.search(
            r"(university of [a-z0-9 .,&'-]+?)(?=\s+school|\s+faculty|\s+department|\s+dept|\s+institute|\s+center|, and\s+| and\s+|/|\(|$)",
            s
        )
        if m:
            return m.group(1).strip()
    
    # 2) '... university' 패턴
    m = re.search(r"([a-z0-9 .,&'-]*?university)", s)
    if m:
        return m.group(1).strip()
    
    # 3) '... college' 패턴
    m = re.search(r"([a-z0-9 .,&'-]*?college)", s)
    if m:
        return m.group(1).strip()
    
    return None

def manual_brand_mapping(s: str) -> str | None:
    """
    university/college 가 명시적이지 않아도
    모두가 아는 브랜드는 본교로 매핑 (같은 애들 → 같은 이름)
    """
    # harvard 계열 (HBS, HLS, Harvard XXX School 등)
    if "harvard" in s:
        return "harvard university"
    # stanford 계열 (GSB 포함)
    if "stanford" in s:
        return "stanford university"
    # wharton → UPenn
    if "wharton" in s:
        return "university of pennsylvania"
    # kellogg → northwestern
    if "kellogg" in s:
        return "northwestern university"
    # ucla anderson → ucla
    if "anderson school" in s and ("ucla" in s or "university of california, los angeles" in s):
        return "university of california, los angeles"
    # UNC Chapel Hill
    if "unc chapel hill" in s:
        return "university of north carolina at chapel hill"

    # 대학 약어/축약 표현 처리
    abbrev_map_exact = {
        "mit": "massachusetts institute of technology",
        "ucsd": "university of california, san diego",
        "uc san diego": "university of california, san diego",
        "ucsb": "university of california, santa barbara",
        "uc irvine": "university of california, irvine",
        "uci": "university of california, irvine",
        "ucla": "university of california, los angeles",
        "ut austin": "university of texas at austin",
        "virginia tech": "virginia polytechnic institute and state university",
        "oxford": "university of oxford",
        "eth zurich": "eth zurich",
        "hec paris": "hec paris",
        "insead": "insead",
        "iit delhi": "indian institute of technology delhi",
        "nit rourkela": "national institute of technology rourkela",
        "nmsu": "new mexico state university",
        "bits": "birla institute of technology and science",
        "ugent": "ghent university",
    }

    # 정확히 일치하는 경우
    if s in abbrev_map_exact:
        return abbrev_map_exact[s]
    
    # 부분 문자열로 들어간 케이스
    abbrev_map_contains = {
        "sun y/buffalo": "university at buffalo",
        "suny/buffalo": "university at buffalo",
        "technion israel": "technion - israel institute of technology",
        "kth": "kth royal institute of technology",
    }
    for key, val in abbrev_map_contains.items():
        if key in s:
            return val

    return None

def flip_university_of(core: str) -> str:
    """
    'university of toronto' → 'toronto university' 로 변환.
    core 는 이미 소문자라고 가정.
    """
    m = re.match(r"university of (.+)", core)
    if m:
        rest = m.group(1).strip()
        rest = re.sub(r"\s+", " ", rest)
        return f"{rest} university"
    return core

def normalize_institution(name: str) -> str:
    """
    최종 정규화:
    - 같은 학교(예: Harvard Business School / Harvard University) → 같은 이름
    - university / college 만 남기고 나머지 꼬리(school, department 등) 제거
    - 'university of X' → 'X university'
    - 전부 소문자
    - london business school, cma ontario 같은 비대학 기관은 그냥 소문자만
    """
    if pd.isna(name) or not str(name).strip():
        return np.nan
    
    s = clean_basic(name)
    
    # 1) 수동 브랜드 매핑 우선 (harvard business school 등)
    brand = manual_brand_mapping(s)
    if brand is not None:
        core = brand  # 이미 'xxx university' 형태
    else:
        # 2) 일반 university / college 패턴 추출
        core = extract_university_like(s)
    
    if core is not None:
        core = core.strip().lower()
        # 'university of X' 패턴이면 X university로 뒤집기
        if core.startswith("university of "):
            core = flip_university_of(core)
        return core
    
    keywords = ["university", "college", "institute", "school", "polytechnic", "tech"]
    if not any(k in s for k in keywords):
        if len(s.split()) <= 2:
            # 사람 이름/잡텍스트 가능성이 높아서 공백으로 처리
            return np.nan
        
    # university / college 없는 애들 (비즈니스 스쿨, 협회 등)은
    # 그냥 깨끗한 소문자 문자열로만 통일
    return s

# 2. 정규화 적용
Degrees_copy["institution_normalized"] = (
    Degrees_copy["institution"]
    .apply(normalize_institution)
    .replace(r"^\s*$", np.nan, regex=True)  
)



In [42]:
# graduated_at 형 변환
Degrees_copy['graduated_at'] = pd.to_datetime(Degrees_copy['graduated_at'])

In [43]:
Degrees_copy

,degrees_p_id,degree_type,subject,institution,graduated_at,cat_degrees_degree_type,degree_level,is_degree_missing,cat_degrees_subject,institution_normalized
0,p:6117,MBA,NaN,NaN,NaT,Master’s degree,3,0,NaN,NaN
1,p:6136,BA,"English, French","Washington University, St. Louis",1990-01-01,Bachelor’s degree,2,0,Arts / Humanities,washington university
2,p:6136,MS,Mass Communication,Boston University,1992-01-01,Master’s degree,3,0,Marketing / Communications,boston university
3,p:6005,MS,Internet Technology,University of Greenwich,2006-01-01,Master’s degree,3,0,Computer Science / Software,greenwich university
4,p:5832,BCS,"Computer Science, Psychology",Rice University,NaT,Bachelor’s degree,2,0,Computer Science / Software,rice university
...,...,...,...,...,...,...,...,...,...,...
109605,p:268589,CPA,NaN,American Institute of Certtified Public Accoun...,NaT,Others,0,0,NaN,american institute of certtified public accoun...
109606,p:268527,MS & BS,Engineering,South China University of Technology,NaT,Bachelor’s degree,2,0,Computer Science / Software,technology university
109607,p:268527,PhD,Engineering,Clarkson University,NaT,Doctorate,4,0,Computer Science / Software,clarkson university
109608,p:268528,B.S.,Electrical Engineering,Colorado State University,NaT,Bachelor’s degree,2,0,Computer Science / Software,colorado state university


In [44]:
# # csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
# Degrees_copy.to_csv('clean_degrees_v3.csv', index=False)
# print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")

## investment 데이터 전처리

In [45]:
# 컬럼 제거
Investments_copy = Investments.copy()
Investments_copy = Investments_copy.drop(columns=['created_at', 'updated_at'])

In [46]:
# funded_object_id, investor_object_id 컬럼명 변경
alter_columns = Investments_copy.rename(columns={
    'id': 'investments_id',
    'funded_object_id': 'invested_c_id',
    'investor_object_id': 'investor_cfp_id'
}, inplace=True)

In [47]:
Investments_copy

,investments_id,funding_round_id,invested_c_id,investor_cfp_id
0,1,1,c:4,f:1
1,2,1,c:4,f:2
2,3,3,c:5,f:4
3,4,4,c:5,f:1
4,5,4,c:5,f:5
...,...,...,...,...
80897,80898,57882,c:285244,p:267312
80898,80899,57912,c:60389,p:268570
80899,80900,57929,c:286183,p:365
80900,80901,57929,c:286183,p:176466


In [48]:
# # csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
# Investments_copy.to_csv('clean_investments_v1.csv', index=False)
# print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")

## ipos 데이터 전처리

In [49]:
# 컬럼 제거
Ipos_copy = Ipos.copy()
Ipos_copy = Ipos_copy.drop(columns=['id', 'source_url', 'source_description', 'created_at', 'updated_at'])

In [50]:
# object_id 컬럼명 변경
alter_columns = Ipos_copy.rename(columns={
    'object_id': 'ipos_c_id'
}, inplace=True)

In [51]:
# public_at 형 변환
Ipos_copy['public_at'] = pd.to_datetime(Ipos_copy['public_at'])

# public_at 결측값 플래그 컬럼 생성
Ipos_copy["is_ipos_public_at_missing"] = Ipos_copy["public_at"].isna().astype(int)

In [52]:
# 상장소 정규화

import numpy as np

# 2) 거래소(상장소)만 추출하는 함수
def extract_exchange_only(stock_symbol: str) -> str:
    if pd.isna(stock_symbol):
        return np.nan

    s = str(stock_symbol).strip()

    # (1) "QPSA (AMEX)" 처럼 괄호에 거래소가 있는 경우 → 괄호 안 우선
    m = re.search(r"\(([^)]+)\)", s)
    if m:
        return m.group(1).strip()

    # (2) 일반적인 "NASDAQ:OTEX" → 콜론 앞부분
    if ":" in s:
        return s.split(":", 1)[0].strip()

    # (3) 콜론도 괄호도 없으면: 일단 전체를 거래소 후보로 반환
    # (데이터에 티커만 들어온 경우가 있어도, 너는 "거래소만 남기기"니까
    #  여기서는 그대로 두고 뒤에서 정규화/검증으로 걸러낼 수 있음)
    return s


# 3) 거래소 추출
Ipos_copy["exchange_raw"] = Ipos_copy["stock_symbol"].apply(extract_exchange_only)

# 4) 대문자/공백 정리
Ipos_copy["exchange_raw"] = Ipos_copy["exchange_raw"].astype(str).str.strip()
Ipos_copy.loc[Ipos_copy["exchange_raw"].isin(["nan", "None", ""]), "exchange_raw"] = np.nan
Ipos_copy["exchange_raw"] = Ipos_copy["exchange_raw"].str.upper()

# 5) 거래소 표준화(정규화) 매핑: "NASDAQ은 다 NASDAQ" 같은 통일 처리
EXCHANGE_MAP = {
    # NASDAQ 계열
    "NASDAQ": "NASDAQ",
    "NAS": "NASDAQ",
    "NASDAQGM": "NASDAQ",
    "NASDAQGS": "NASDAQ",
    "NASDAQCM": "NASDAQ",
    "NASDAQ NMS": "NASDAQ",
    "NASDAQ NMQ": "NASDAQ",

    # NYSE 계열
    "NYSE": "NYSE",
    "NYQ": "NYSE",
    "NYE": "NYSE",
    "NYA": "NYSE",

    # AMEX 계열
    "AMEX": "AMEX",
    "ASE": "AMEX",

    # OTC / 장외
    "OTC": "OTC",
    "OTCBB": "OTC",
    "OTCQB": "OTC",
    "OTCQX": "OTC",
    "OTCM": "OTC",
    "OTCMKTS": "OTC",
    "PINK": "OTC",

    # London (필요시 통일)
    "LSE": "LSE",
    "LON": "LSE",
    "AIM": "AIM",   # AIM은 LSE의 성장시장이라 분리 유지(원하면 "LSE"로 합쳐도 됨)

    # Tokyo (필요시 통일)
    "TSE": "TSE",
    "TYO": "TSE",

    # Canada / Australia / etc.
    "TSX": "TSX",
    "TSXV": "TSX",
    "ASX": "ASX",
    "SGX": "SGX",
}

Ipos_copy["exchange_norm"] = Ipos_copy["exchange_raw"].map(EXCHANGE_MAP).fillna(Ipos_copy["exchange_raw"])

# 6) 최종: "티커 삭제하고 거래소만" 컬럼 만들기
Ipos_copy["stock_normalized"] = Ipos_copy["exchange_norm"]

# 중간 컬럼 exchange_norm 삭제
Ipos_copy = Ipos_copy.drop(columns=["exchange_raw", "exchange_norm"])

In [53]:
Ipos_copy['stock_symbol'].value_counts()

stock_symbol
NASDAQ:OTEX    2
NYSE:ABT       2
NYSE:ACT       2
NASDAQ:RRD     2
NASDAQ:WWVY    2
              ..
NASDAQ:EEFT    1
NASDAQ:CHTR    1
NYSE:SLB       1
ATO.PA         1
NASDAQ:KIN     1
Name: count, Length: 1247, dtype: int64

In [54]:
# ipo valuation_amount, valuation_amount nan 값 대체, 비공개 플래그 생성
import numpy as np

# valuation_amount
# 0값을 nan으로 대체
Ipos_copy["valuation_amount"] = Ipos_copy["valuation_amount"].replace(0, np.nan)

# 비공개 플래그 변수 생성 → is_ipos_valuation_private
Ipos_copy["is_ipos_valuation_private"] = np.nan
Ipos_copy["is_ipos_valuation_private"] = np.where(# 비공개냐
    Ipos_copy["valuation_amount"].isna(), #비공개(nan)
    1,  #비공개(nan)
    0   #공개(값이 있다)
)
display(Ipos_copy["is_ipos_valuation_private"].value_counts())

is_ipos_valuation_private
1    1151
0     108
Name: count, dtype: int64

In [55]:
# Load Files (경로확인 후 실행!)
usd_aud = pd.read_csv("../환율 계산/USD_AUD_rate.csv")
usd_cad = pd.read_csv("../환율 계산/USD_CAD_rate.csv")
usd_eur = pd.read_csv("../환율 계산/USD_EUR_rate.csv")
usd_gbp = pd.read_csv("../환율 계산/USD_GBP_rate.csv")
usd_jpy = pd.read_csv("../환율 계산/USD_JPY_rate.csv")
usd_sek = pd.read_csv("../환율 계산/USD_SEK_rate.csv")

print("="*60)
print("환율 Dataset 로드 완료!")
print("="*60)

환율 Dataset 로드 완료!


In [56]:
# 각 환율코드별 날짜 min&max 값 찾기 
print("="*60)
print("AUD")
cond1 = (Ipos_copy['valuation_currency_code'] =="AUD")
print(Ipos_copy.loc[cond1, "public_at"].min())
print(Ipos_copy.loc[cond1, "public_at"].max())

print("="*60)
print("CAD")
cond2 = (Ipos_copy['valuation_currency_code'] =="CAD")
print(Ipos_copy.loc[cond2, "public_at"].min())
print(Ipos_copy.loc[cond2, "public_at"].max())

print("="*60)
print("EUR")
cond3 = (Ipos_copy['valuation_currency_code'] =="EUR")
print(Ipos_copy.loc[cond3, "public_at"].min())
print(Ipos_copy.loc[cond3, "public_at"].max())

print("="*60)
print("GBP")
cond4 = (Ipos_copy['valuation_currency_code'] =="GBP")
print(Ipos_copy.loc[cond4, "public_at"].min())
print(Ipos_copy.loc[cond4, "public_at"].max())

print("="*60)
print("JPY")
cond5 = (Ipos_copy['valuation_currency_code'] =="JPY")
print(Ipos_copy.loc[cond5, "public_at"].min())
print(Ipos_copy.loc[cond5, "public_at"].max())

print("="*60)
print("SEK")
cond6 = (Ipos_copy['valuation_currency_code'] =="SEK")
print(Ipos_copy.loc[cond6, "public_at"].min())
print(Ipos_copy.loc[cond6, "public_at"].max())

AUD
NaT
NaT
CAD
1989-06-23 00:00:00
1989-06-23 00:00:00
EUR
2000-02-22 00:00:00
2000-02-22 00:00:00
GBP
2013-05-21 00:00:00
2013-05-21 00:00:00
JPY
1987-06-16 00:00:00
2008-12-17 00:00:00
SEK
NaT
NaT


In [57]:
# 환율 계산 함수
def cal_rate(code):

    if code=="CAD":
        cond2 = (usd_cad["observation_date"]>="1989-06-23") & (usd_cad["observation_date"]<="1989-06-23")
        result = usd_cad.loc[cond2, "DEXCAUS"].mean()

    if code=="EUR":
        cond3 = (usd_eur["observation_date"]>="2000-02-22") & (usd_eur["observation_date"]<="2000-02-22")
        result = usd_eur.loc[cond3, "DEXUSEU"].mean()
    

    if code=="GBP":
        cond4 = (usd_gbp["observation_date"]>="2013-05-21") & (usd_gbp["observation_date"]<="2013-05-21")
        result = usd_gbp.loc[cond4, "DEXUSUK"].mean()

    if code=="JPY":
        cond5 = (usd_jpy["observation_date"]>="1987-06-16") & (usd_jpy["observation_date"]<="2008-12-17")
        result = usd_jpy.loc[cond5, "DEXJPUS"].mean()
    
    return result

In [58]:
# ipo valuation_amount 환율 적용 

cad_mean = cal_rate("CAD")
eur_mean = cal_rate("EUR")
gbp_mean = cal_rate("GBP")
jpy_mean = cal_rate("JPY")
usd_mean = 1.0

Ipos_copy["valuation_amount_usd"] = np.nan
Ipos_copy["valuation_currency_rate"] = np.nan

avg_rate = {
    "CAD": cad_mean,   # 예: 0.85  (U.S. $ to 1 CAD 이면 곱셈)
    "EUR": eur_mean,
    "GBP": gbp_mean,
    "JPY": jpy_mean,   # 예: 110.0 (JPY to 1 USD 이면 나눗셈)
    "USD": 1.0         # 이미 USD인 경우 그대로 사용
}

for cur, r in avg_rate.items():
    cond = Ipos_copy["valuation_currency_code"] == cur
    
    # 방향에 따라 곱하거나 나누기
    if cur in ["CAD", "EUR", "GBP"]:   # "U.S. $ to 1 통화" 타입
        Ipos_copy.loc[cond, "valuation_amount_usd"] = (
            Ipos_copy.loc[cond, "valuation_amount"] * r
        )
        Ipos_copy.loc[cond, "valuation_currency_rate"] = r
    elif cur in ["JPY"]:               # "통화 to 1 U.S. $" 타입
        Ipos_copy.loc[cond, "valuation_amount_usd"] = (
            Ipos_copy.loc[cond, "valuation_amount"] / r
        )
        Ipos_copy.loc[cond, "valuation_currency_rate"] = r
    else:  # USD 등
        Ipos_copy.loc[cond, "valuation_amount_usd"] = Ipos_copy.loc[cond, "valuation_amount"]
        Ipos_copy.loc[cond, "valuation_currency_rate"] = r


#확인
cond1 = (Ipos_copy["valuation_currency_code"] != "USD")
cond2 = (Ipos_copy["valuation_amount"] > 0)
display(Ipos_copy.loc[cond1 & cond2, ["ipos_c_id","valuation_currency_code", "valuation_currency_rate", "valuation_amount", "valuation_amount_usd"]])

,ipos_c_id,valuation_currency_code,valuation_currency_rate,valuation_amount,valuation_amount_usd
67,c:15609,JPY,112.0873,1.089600e+11,9.720994e+08
295,c:9580,EUR,NaN,4.400000e+07,NaN
440,c:9636,JPY,112.0873,1.179000e+05,1.051859e+03
843,c:66694,GBP,1.5170,3.000000e+07,4.551000e+07


In [59]:
# ipo raised_amount, raised_amount nan 값 대체, 비공개 플래그 생성
import numpy as np

# raised_amount
#0값을 nan으로 대체
Ipos_copy["raised_amount"] = Ipos_copy["raised_amount"].replace(0, np.nan)

# 비공개 플래그 변수 생성 → is_ipos_raised_private
Ipos_copy["is_ipos_raised_private"] = np.nan
Ipos_copy["is_ipos_raised_private"] = np.where(# 비공개냐
    Ipos_copy["raised_amount"].isna(), #비공개(nan)
    1,  #비공개(nan)
    0   #공개(값이 있다)
)
display(Ipos_copy["is_ipos_raised_private"].value_counts())

is_ipos_raised_private
1    1121
0     138
Name: count, dtype: int64

In [60]:
# 각 환율코드별 날짜 min&max 값 찾기 
print("="*60)
print("AUD")
cond1 = (Ipos_copy['raised_currency_code'] =="AUD")
print(Ipos_copy.loc[cond1, "public_at"].min())
print(Ipos_copy.loc[cond1, "public_at"].max())

print("="*60)
print("CAD")
cond2 = (Ipos_copy['raised_currency_code'] =="CAD")
print(Ipos_copy.loc[cond2, "public_at"].min())
print(Ipos_copy.loc[cond2, "public_at"].max())

print("="*60)
print("EUR")
cond3 = (Ipos_copy['raised_currency_code'] =="EUR")
print(Ipos_copy.loc[cond3, "public_at"].min())
print(Ipos_copy.loc[cond3, "public_at"].max())

print("="*60)
print("GBP")
cond4 = (Ipos_copy['raised_currency_code'] =="GBP")
print(Ipos_copy.loc[cond4, "public_at"].min())
print(Ipos_copy.loc[cond4, "public_at"].max())

print("="*60)
print("JPY")
cond5 = (Ipos_copy['raised_currency_code'] =="JPY")
print(Ipos_copy.loc[cond5, "public_at"].min())
print(Ipos_copy.loc[cond5, "public_at"].max())

print("="*60)
print("SEK")
cond6 = (Ipos_copy['raised_currency_code'] =="SEK")
print(Ipos_copy.loc[cond6, "public_at"].min())
print(Ipos_copy.loc[cond6, "public_at"].max())

AUD
NaT
NaT
CAD
2011-02-01 00:00:00
2011-02-01 00:00:00
EUR
2007-06-22 00:00:00
2007-06-22 00:00:00
GBP
2013-05-21 00:00:00
2013-05-21 00:00:00
JPY
1987-06-16 00:00:00
2002-05-23 00:00:00
SEK
NaT
NaT


In [61]:
# 환율 계산 함수
def cal_rate(code):

    if code=="CAD":
        cond2 = (usd_cad["observation_date"]>="2011-02-01") & (usd_cad["observation_date"]<="2011-02-01")
        result = usd_cad.loc[cond2, "DEXCAUS"].mean()

    if code=="EUR":
        cond3 = (usd_eur["observation_date"]>="2007-06-22") & (usd_eur["observation_date"]<="2007-06-22")
        result = usd_eur.loc[cond3, "DEXUSEU"].mean()
    

    if code=="GBP":
        cond4 = (usd_gbp["observation_date"]>="2013-05-21") & (usd_gbp["observation_date"]<="2013-05-21")
        result = usd_gbp.loc[cond4, "DEXUSUK"].mean()

    if code=="JPY":
        cond5 = (usd_jpy["observation_date"]>="1987-06-16") & (usd_jpy["observation_date"]<="2002-05-23")
        result = usd_jpy.loc[cond5, "DEXJPUS"].mean()
    
    return result

In [62]:
Ipos_copy

,ipo_id,ipos_c_id,valuation_amount,valuation_currency_code,raised_amount,raised_currency_code,public_at,stock_symbol,is_ipos_public_at_missing,stock_normalized,is_ipos_valuation_private,valuation_amount_usd,valuation_currency_rate,is_ipos_raised_private
0,1,c:1654,NaN,USD,NaN,USD,1980-12-19,NASDAQ:AAPL,0,NASDAQ,1,NaN,1.0,1
1,2,c:1242,NaN,USD,NaN,NaN,1986-03-13,NASDAQ:MSFT,0,NASDAQ,1,NaN,1.0,1
2,3,c:342,NaN,USD,NaN,NaN,1969-06-09,NYSE:DIS,0,NYSE,1,NaN,1.0,1
3,4,c:59,NaN,USD,NaN,NaN,2004-08-25,NASDAQ:GOOG,0,NASDAQ,1,NaN,1.0,1
4,5,c:317,1.000000e+11,USD,NaN,NaN,1997-05-01,NASDAQ:AMZN,0,NASDAQ,0,1.000000e+11,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,1372,c:267859,NaN,USD,NaN,USD,NaT,NYSE:EBS,1,NYSE,1,NaN,1.0,1
1255,1373,c:39330,NaN,USD,7.250000e+08,USD,2013-12-11,NYSE:ARMK,0,NYSE,1,NaN,1.0,0
1256,1374,c:220208,NaN,USD,2.350000e+09,USD,2008-12-08,NYSE:HLT,0,NYSE,1,NaN,1.0,0
1257,1375,c:243568,NaN,USD,NaN,USD,NaT,OTC:BCLI,1,OTC,1,NaN,1.0,1


In [63]:
# ipo ipo_raised_amount 환율 적용 

cad_mean = cal_rate("CAD")
eur_mean = cal_rate("EUR")
gbp_mean = cal_rate("GBP")
jpy_mean = cal_rate("JPY")
usd_mean = 1.0

Ipos_copy["ipo_raised_amount_usd"] = np.nan
Ipos_copy["ipo_raised_currency_rate"] = np.nan

avg_rate = {
    "CAD": cad_mean,   # 예: 0.85  (U.S. $ to 1 CAD 이면 곱셈)
    "EUR": eur_mean,
    "GBP": gbp_mean,
    "JPY": jpy_mean,   # 예: 110.0 (JPY to 1 USD 이면 나눗셈)
    "USD": 1.0         # 이미 USD인 경우 그대로 사용
}

for cur, r in avg_rate.items():
    cond = Ipos_copy["raised_currency_code"] == cur
    
    # 방향에 따라 곱하거나 나누기
    if cur in ["CAD", "EUR", "GBP"]:   # "U.S. $ to 1 통화" 타입
        Ipos_copy.loc[cond, "ipo_raised_amount_usd"] = (
            Ipos_copy.loc[cond, "raised_amount"] * r
        )
        Ipos_copy.loc[cond, "ipo_raised_currency_rate"] = r
    elif cur in ["JPY"]:               # "통화 to 1 U.S. $" 타입
        Ipos_copy.loc[cond, "ipo_raised_amount_usd"] = (
            Ipos_copy.loc[cond, "raised_amount"] / r
        )
        Ipos_copy.loc[cond, "ipo_raised_currency_rate"] = r
    else:  # USD 등
        Ipos_copy.loc[cond, "ipo_raised_amount_usd"] = Ipos_copy.loc[cond, "raised_amount"]
        Ipos_copy.loc[cond, "ipo_raised_currency_rate"] = r


#확인
cond1 = (Ipos_copy["raised_currency_code"] != "USD")
cond2 = (Ipos_copy["raised_amount"] > 0)
display(Ipos_copy.loc[cond1 & cond2, ["ipos_c_id","raised_currency_code", "ipo_raised_currency_rate", "raised_amount", "ipo_raised_amount_usd"]])

,ipos_c_id,raised_currency_code,ipo_raised_currency_rate,raised_amount,ipo_raised_amount_usd
15,c:108,EUR,1.3438,2.541110e+05,3.414744e+05
29,c:1863,EUR,1.3438,3.570000e+07,4.797366e+07
843,c:66694,GBP,1.5170,3.880000e+06,5.885960e+06
998,c:141691,CAD,0.9921,1.130000e+07,1.121073e+07
1200,c:277665,JPY,NaN,1.100000e+09,NaN
1230,c:27295,GBP,1.5170,2.440000e+10,3.701480e+10


In [64]:
Ipos_copy

,ipo_id,ipos_c_id,valuation_amount,valuation_currency_code,raised_amount,raised_currency_code,public_at,stock_symbol,is_ipos_public_at_missing,stock_normalized,is_ipos_valuation_private,valuation_amount_usd,valuation_currency_rate,is_ipos_raised_private,ipo_raised_amount_usd,ipo_raised_currency_rate
0,1,c:1654,NaN,USD,NaN,USD,1980-12-19,NASDAQ:AAPL,0,NASDAQ,1,NaN,1.0,1,NaN,1.0
1,2,c:1242,NaN,USD,NaN,NaN,1986-03-13,NASDAQ:MSFT,0,NASDAQ,1,NaN,1.0,1,NaN,NaN
2,3,c:342,NaN,USD,NaN,NaN,1969-06-09,NYSE:DIS,0,NYSE,1,NaN,1.0,1,NaN,NaN
3,4,c:59,NaN,USD,NaN,NaN,2004-08-25,NASDAQ:GOOG,0,NASDAQ,1,NaN,1.0,1,NaN,NaN
4,5,c:317,1.000000e+11,USD,NaN,NaN,1997-05-01,NASDAQ:AMZN,0,NASDAQ,0,1.000000e+11,1.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,1372,c:267859,NaN,USD,NaN,USD,NaT,NYSE:EBS,1,NYSE,1,NaN,1.0,1,NaN,1.0
1255,1373,c:39330,NaN,USD,7.250000e+08,USD,2013-12-11,NYSE:ARMK,0,NYSE,1,NaN,1.0,0,7.250000e+08,1.0
1256,1374,c:220208,NaN,USD,2.350000e+09,USD,2008-12-08,NYSE:HLT,0,NYSE,1,NaN,1.0,0,2.350000e+09,1.0
1257,1375,c:243568,NaN,USD,NaN,USD,NaT,OTC:BCLI,1,OTC,1,NaN,1.0,1,NaN,1.0


In [65]:
# # csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
# Ipos_copy.to_csv('clean_ipos_v2.csv', index=False)
# print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")